In [1]:
import pandas as pd, numpy as np, datetime as dt, requests
import pandas as pd, matplotlib.pyplot as plt, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
#import regex as re
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor 
from sklearn.model_selection import GridSearchCV

In [2]:
my_df = pd.read_csv("C:\\Users\\jampa\\OneDrive\\Desktop\\UMass Study\\AF478\\all_data2.csv")

my_df['Date']=pd.to_datetime(my_df['Date'])
my_df['year']=my_df['Date'].dt.year
my_df['month']=my_df['Date'].dt.month
my_df['day']=my_df['Date'].dt.day
my_df.sort_values(by=['symbol','year','month','day'],inplace=True)

In [3]:
my_df.reset_index(drop=True,inplace=True)
my_df.rename(columns = {'Adj Close':'adj_close'}, inplace = True)

In [4]:
my_df['prev_price']=my_df.groupby('symbol')['adj_close'].shift(1)
my_df['daily_return']=(  my_df['adj_close']-my_df['prev_price'] )/ my_df['prev_price']

my_df['avrg_return_daily'] = my_df.groupby(['symbol','year','month'])['daily_return'].transform('mean')
my_df['std_return_daily'] = my_df.groupby(['symbol','year','month'])['daily_return'].transform('std')
my_df['skew_return_daily'] = my_df.groupby(['symbol','year','month'])['daily_return'].transform('skew')
my_df['Sharpe']=my_df['avrg_return_daily']/my_df['std_return_daily']
my_df

,Date,Open,High,Low,Close,adj_close,Volume,symbol,year,month,day,prev_price,daily_return,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe
0,1999-11-18,32.546494,35.765381,28.612303,31.473534,26.740847,62546380.0,A,1999,11,18,NaN,NaN,-0.004237,0.063538,-0.225378,-0.066687
1,1999-11-19,30.713518,30.758226,28.478184,28.880545,24.537760,15234146.0,A,1999,11,19,26.740847,-0.082387,-0.004237,0.063538,-0.225378,-0.066687
2,1999-11-22,29.551144,31.473534,28.657009,31.473534,26.740847,6577870.0,A,1999,11,22,24.537760,0.089784,-0.004237,0.063538,-0.225378,-0.066687
3,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.309858,5975611.0,A,1999,11,23,26.740847,-0.090909,-0.004237,0.063538,-0.225378,-0.066687
4,1999-11-24,28.701717,29.998213,28.612303,29.372318,24.955591,4843231.0,A,1999,11,24,24.309858,0.026563,-0.004237,0.063538,-0.225378,-0.066687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216416,2023-12-05,181.289993,182.039993,178.860001,179.649994,179.649994,1425900.0,ZTS,2023,12,5,182.119995,-0.013562,0.010151,0.013216,-0.984151,0.768087
4216417,2023-12-06,180.750000,183.179993,180.639999,182.000000,182.000000,1552900.0,ZTS,2023,12,6,179.649994,0.013081,0.010151,0.013216,-0.984151,0.768087
4216418,2023-12-07,182.460007,182.600006,180.509995,181.830002,181.830002,1088600.0,ZTS,2023,12,7,182.000000,-0.000934,0.010151,0.013216,-0.984151,0.768087
4216419,2023-12-08,182.080002,184.850006,181.470001,184.600006,184.600006,1474600.0,ZTS,2023,12,8,181.830002,0.015234,0.010151,0.013216,-0.984151,0.768087


In [5]:
df=(my_df.groupby(['symbol','year','month']).last().reset_index()).drop(columns=['daily_return','Open','High','Low','Volume','day','prev_price'])


In [6]:
df['prev_price']=df.groupby('symbol')['adj_close'].shift(1)
df['return']=(  df['adj_close']-df['prev_price'] )/df['prev_price']


In [7]:
df['next_month_return']=df.groupby('symbol')['return'].shift(-1)


In [8]:
df

,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,prev_price,return,next_month_return
0,A,1999,11,1999-11-30,30.177038,25.639307,-0.004237,0.063538,-0.225378,-0.066687,NaN,NaN,0.832592
1,A,1999,12,1999-12-31,55.302216,46.986397,0.029614,0.061724,1.181711,0.479782,25.639307,0.832592,-0.143896
2,A,2000,1,2000-01-31,47.344421,40.225224,-0.006968,0.040263,0.382874,-0.173071,46.986397,-0.143896,0.569405
3,A,2000,2,2000-02-29,74.302574,63.129662,0.024397,0.059944,1.181116,0.407004,40.225224,0.569405,0.001204
4,A,2000,3,2000-03-31,74.391991,63.205650,0.005110,0.113234,3.366080,0.045123,63.129662,0.001204,-0.147837
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201361,ZTS,2023,8,2023-08-31,190.509995,190.054947,0.000684,0.016494,1.283727,0.041494,187.640732,0.012866,-0.086767
201362,ZTS,2023,9,2023-09-29,173.979996,173.564438,-0.004461,0.011811,-0.679612,-0.377698,190.054947,-0.086767,-0.095437
201363,ZTS,2023,10,2023-10-31,157.000000,157.000000,-0.004472,0.012649,-0.546997,-0.353544,173.564438,-0.095437,0.125287
201364,ZTS,2023,11,2023-11-30,176.669998,176.669998,0.005804,0.018882,0.931149,0.307381,157.000000,0.125287,0.072706


In [10]:
df['cum_ret6']= df.groupby('symbol')['return'].rolling(6).apply(lambda x: np.prod(1+x)-1).reset_index(drop=True) 
df.head(10)

,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,prev_price,return,next_month_return,cum_ret6
0,A,1999,11,1999-11-30,30.177038,25.639307,-0.004237,0.063538,-0.225378,-0.066687,NaN,NaN,0.832592,NaN
1,A,1999,12,1999-12-31,55.302216,46.986397,0.029614,0.061724,1.181711,0.479782,25.639307,0.832592,-0.143896,NaN
2,A,2000,1,2000-01-31,47.344421,40.225224,-0.006968,0.040263,0.382874,-0.173071,46.986397,-0.143896,0.569405,NaN
3,A,2000,2,2000-02-29,74.302574,63.129662,0.024397,0.059944,1.181116,0.407004,40.225224,0.569405,0.001204,NaN
4,A,2000,3,2000-03-31,74.391991,63.205650,0.005110,0.113234,3.366080,0.045123,63.129662,0.001204,-0.147837,NaN
5,A,2000,4,2000-04-28,63.394135,53.861515,-0.005711,0.075022,0.212424,-0.076128,63.205650,-0.147837,-0.169252,NaN
6,A,2000,5,2000-05-31,52.664520,44.745338,-0.005072,0.085288,1.083793,-0.059472,53.861515,-0.169252,0.001698,0.745185
7,A,2000,6,2000-06-30,52.753933,44.821301,0.001576,0.056377,0.381416,0.027958,44.745338,0.001698,-0.447458,-0.046079
8,A,2000,7,2000-07-31,29.148785,24.765671,-0.026354,0.072656,-2.054010,-0.362716,44.821301,-0.447458,0.496933,-0.384325
9,A,2000,8,2000-08-31,43.633762,37.072540,0.018962,0.053768,2.345232,0.352655,24.765671,0.496933,-0.197746,-0.412756


In [11]:
df['cum_ret12']= df.groupby('symbol')['return'].rolling(12).apply(lambda x: np.prod(1+x)-1).reset_index(drop=True) 
df

,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,prev_price,return,next_month_return,cum_ret6,cum_ret12
0,A,1999,11,1999-11-30,30.177038,25.639307,-0.004237,0.063538,-0.225378,-0.066687,NaN,NaN,0.832592,NaN,NaN
1,A,1999,12,1999-12-31,55.302216,46.986397,0.029614,0.061724,1.181711,0.479782,25.639307,0.832592,-0.143896,NaN,NaN
2,A,2000,1,2000-01-31,47.344421,40.225224,-0.006968,0.040263,0.382874,-0.173071,46.986397,-0.143896,0.569405,NaN,NaN
3,A,2000,2,2000-02-29,74.302574,63.129662,0.024397,0.059944,1.181116,0.407004,40.225224,0.569405,0.001204,NaN,NaN
4,A,2000,3,2000-03-31,74.391991,63.205650,0.005110,0.113234,3.366080,0.045123,63.129662,0.001204,-0.147837,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201361,ZTS,2023,8,2023-08-31,190.509995,190.054947,0.000684,0.016494,1.283727,0.041494,187.640732,0.012866,-0.086767,0.145743,0.227855
201362,ZTS,2023,9,2023-09-29,173.979996,173.564438,-0.004461,0.011811,-0.679612,-0.377698,190.054947,-0.086767,-0.095437,0.049850,0.183626
201363,ZTS,2023,10,2023-10-31,157.000000,157.000000,-0.004472,0.012649,-0.546997,-0.353544,173.564438,-0.095437,0.125287,-0.102731,0.050750
201364,ZTS,2023,11,2023-11-30,176.669998,176.669998,0.005804,0.018882,0.931149,0.307381,157.000000,0.125287,0.072706,0.088783,0.156621


In [12]:
features=['Close','avrg_return_daily','std_return_daily','skew_return_daily','Sharpe','return','cum_ret6','cum_ret12']
_df=(df[ (df['year']>=2010) & (df['year']<=2022)  ]).dropna()

X=_df[features]

y=(_df['next_month_return']).reset_index()
y.drop(columns=['index'],inplace=True)
X

,Close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,return,cum_ret6,cum_ret12
122,20.050072,-0.005306,0.014345,-1.057651,-0.369864,-0.097844,0.207149,0.550332
123,22.503576,0.006162,0.012060,0.795518,0.511000,0.122369,0.225078,1.268206
124,24.599428,0.003922,0.009486,0.943496,0.413428,0.093134,0.235717,1.237475
125,25.937054,0.002628,0.014793,0.484652,0.177654,0.054376,0.465642,0.985761
126,23.147352,-0.005176,0.032369,0.345697,-0.159907,-0.107557,0.118949,0.775096
...,...,...,...,...,...,...,...,...
201349,156.529999,-0.006552,0.015242,0.131837,-0.429848,-0.142536,-0.188785,-0.229898
201350,148.289993,-0.002436,0.016930,0.549872,-0.143875,-0.052642,-0.210869,-0.231280
201351,150.779999,0.001047,0.017897,-0.199821,0.058510,0.018952,-0.145965,-0.297473
201352,154.139999,0.001700,0.036468,-1.137458,0.046623,0.022284,-0.094653,-0.300707


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8,test_size=0.25)
_avrg=X_train.mean()
_std=X_train.std()
X_train=(X_train - _avrg)/ (_std)
X_test=(X_test - _avrg)/ (_std)


In [15]:
from sklearn.model_selection import GridSearchCV

## Problem 1

In [16]:
parameter={'alpha':[a for a in range(100,10000,100)]}
_model=Ridge()
my_ridge=GridSearchCV(_model, param_grid=parameter, return_train_score=True, cv=5).fit(X_train,y_train)


In [17]:
my_ridge.best_estimator_

Ridge(alpha=1800)

In [18]:
ridge_model=Ridge(alpha=1800,).fit(X_train,y_train)
print(ridge_model.score(X_train,y_train),ridge_model.score(X_test,y_test))


0.014495703852916209 0.019923008399222786


In [19]:
print(mean_squared_error(y_train,ridge_model.predict(X_train)))
print(mean_squared_error(y_test,ridge_model.predict(X_test)))

0.006961683309181819
0.007102504243078973


## Problem 2

In [23]:
parameters={'max_depth':[3,4,6],'min_samples_leaf':[6,20],'min_samples_split':[20,50],'max_leaf_nodes':[6,12]}
_model=DecisionTreeRegressor()
my_dfr=GridSearchCV(_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train,y_train)


In [24]:
my_dfr.best_estimator_

DecisionTreeRegressor(max_depth=6, max_leaf_nodes=12, min_samples_leaf=6,
                      min_samples_split=20)

In [25]:
dfr_model=DecisionTreeRegressor(max_depth=6,min_samples_leaf=6,min_samples_split=20,max_leaf_nodes=20).fit(X_train,y_train)
print(dfr_model.score(X_train,y_train),dfr_model.score(X_test,y_test))

0.040396822231132834 0.022838735924750275


In [26]:
print(mean_squared_error(y_train,dfr_model.predict(X_train)))
print(mean_squared_error(y_test,dfr_model.predict(X_test)))

0.006778715681128109
0.007081374304003577


## Problem 3

In [27]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
parameter={'n_estimators':[50,60,100],'max_depth':[3,4,6],'min_samples_leaf':[6,20],'min_samples_split':[20,50],'max_leaf_nodes':[6,12]}
_model=RandomForestRegressor()
my_rfr=GridSearchCV(_model, param_grid=parameter, return_train_score=True, cv=5).fit(X_train,y_train)

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

...

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for

In [26]:
my_rfr.best_estimator_

RandomForestRegressor(max_depth=6, max_leaf_nodes=12, min_samples_leaf=6,
                      min_samples_split=20)

In [27]:
rfr_model=RandomForestRegressor(n_estimators=60,max_depth=3,min_samples_leaf=20,min_samples_split=20,max_leaf_nodes=6).fit(X_train,y_train)
print(rfr_model.score(X_train,y_train),rfr_model.score(X_test,y_test))

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.027233520978352077 0.024843975208350577


In [28]:
print(mean_squared_error(y_train,rfr_model.predict(X_train)))
print(mean_squared_error(y_test,rfr_model.predict(X_test)))

0.0068717023225699425
0.007066842567576525


## Problem 4 

In [29]:
from sklearn.neural_network import MLPRegressor


In [31]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

parameters = {
    'solver': ['lbfgs'],  # Wrap 'lbfgs' in a list
    'alpha': [0],
    'hidden_layer_sizes': [(100), (10, 10), (10, 20)],
    'activation': ['identity', 'logistic', 'relu'],
    'max_iter': [1000]
}

_model = MLPRegressor()
my_mlp = GridSearchCV(_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train, y_train)


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

In [30]:
my_mlp.best_estimator_

NameError: name 'my_mlp' is not defined

In [31]:
mlp_model=MLPRegressor(solver='lbfgs',alpha=0,hidden_layer_sizes=(100),activation='relu',max_iter=1000).fit(X_train,y_train)
print(mlp_model.score(X_train,y_train),mlp_model.score(X_test,y_test))

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.05106111123417778 0.0473682732398939


In [32]:
print(mean_squared_error(y_train,mlp_model.predict(X_train)))
print(mean_squared_error(y_test,mlp_model.predict(X_test)))

0.00670338226751739
0.006903611593160817


In [33]:
parameters = {
    'solver': ['sgd', 'adam'],
    'alpha': [0],
    'hidden_layer_sizes': [(100), (10, 20), (10, 10)],
    'activation': ['identity', 'logistic', 'relu'],
    'max_iter': [1000],
    'learning_rate_init': [0.01, 0.1],  # Wrap values in a list
    'batch_size': [20, 50]  # Wrap values in a list
}
_model=MLPRegressor()
my_mlp1=GridSearchCV(_model,param_grid=parameters, return_train_score=True, cv=5).fit(X_train,y_train)


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Us

...

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

In [ ]:
my_mlp1.best_estimator_

In [38]:
mlp1_model=MLPRegressor(solver='sgd',alpha=0,hidden_layer_sizes=(10,10),activation='relu',max_iter=1000,learning_rate_init=0.01,batch_size=200).fit(X_train,y_train)
print(mlp1_model.score(X_train,y_train),mlp1_model.score(X_test,y_test))

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.00830972656864748 0.009907563050878143


In [39]:
print(mean_squared_error(y_train,mlp1_model.predict(X_train)))
print(mean_squared_error(y_test,mlp1_model.predict(X_test)))

0.007005381560908613
0.007175085013459835


## Problem 6

In [34]:
New_X=X.copy()

for j,col1 in enumerate(list(X.columns)):
    for col2 in list(X.columns)[j+1:]:
        New_X[col1+'*'+col2]=X[col1]*X[col2]

for col in list(X.columns):
    New_X[col+'^2']=X[col]*X[col]


In [35]:
X_train,X_test,y_train,y_test=train_test_split(New_X,y,random_state=8,test_size=0.25)
_avg=X_train.mean()
_std=X_train.std()
X_train = (X_train - _avg)/_std
X_test = (X_test - _avg)/_std

#### Ridge

In [36]:
parameter = {'alpha': [a for a in range(1000, 200000, 100)]}
_model = Ridge()
my_ridge = GridSearchCV(_model, param_grid=parameter, return_train_score=True, cv=5).fit(X_train, y_train)

# Retrieve the best alpha value
best_alpha = my_ridge.best_params_['alpha']

# Create the Ridge model using the best alpha
ridge_model = Ridge(alpha=best_alpha).fit(X_train, y_train)

# Evaluate the model
print("Train R2 Score:", ridge_model.score(X_train, y_train))
print("Test R2 Score:", ridge_model.score(X_test, y_test))

print("Train Mean Squared Error:", mean_squared_error(y_train, ridge_model.predict(X_train)))
print("Test Mean Squared Error:", mean_squared_error(y_test, ridge_model.predict(X_test)))


Train R2 Score: 0.02764471255537959
Test R2 Score: 0.04124079333163466
Train Mean Squared Error: 0.006868797631489589
Test Mean Squared Error: 0.006948016729104993


#### DecisionTreeRegressor

In [37]:
parameters={'max_depth':[3],'min_samples_leaf':[6],'min_samples_split':[20],'max_leaf_nodes':[6]}
_model=DecisionTreeRegressor()
my_dfr=GridSearchCV(_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train,y_train)

In [38]:
my_dfr.best_estimator_

DecisionTreeRegressor(max_depth=3, max_leaf_nodes=6, min_samples_leaf=6,
                      min_samples_split=20)

In [39]:
dfr_model=DecisionTreeRegressor(max_depth=3,min_samples_leaf=6,min_samples_split=20,max_leaf_nodes=6).fit(X_train,y_train)
print(dfr_model.score(X_train,y_train),dfr_model.score(X_test,y_test))

print(mean_squared_error(y_train,dfr_model.predict(X_train)))
print(mean_squared_error(y_test,dfr_model.predict(X_test)))

0.024909655023910626 0.02854468400851129
0.006888118302582471
0.007040023960282612


#### RandomForestRegressor

In [40]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

parameter = {
    'max_depth': [3, 4, 6],
    'min_samples_leaf': [6, 20],
    'min_samples_split': [20, 50],
    'max_leaf_nodes': [6, 12]
}

_model = DecisionTreeRegressor()  # Use DecisionTreeRegressor instead of RandomForestRegressor
my_tree = GridSearchCV(_model, param_grid=parameter, return_train_score=True, cv=5).fit(X_train, y_train)


In [41]:
my_rfr.best_estimator_

RandomForestRegressor(max_depth=6, max_leaf_nodes=12, min_samples_leaf=6,
                      min_samples_split=50, n_estimators=50)

In [42]:
rfr_model=RandomForestRegressor(n_estimators=5,max_depth=6,min_samples_leaf=6,min_samples_split=20,max_leaf_nodes=12).fit(X_train,y_train)
print(rfr_model.score(X_train,y_train),rfr_model.score(X_test,y_test))

print(mean_squared_error(y_train,rfr_model.predict(X_train)))
print(mean_squared_error(y_test,rfr_model.predict(X_test)))

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.04054794686066643 0.02815991088087577
0.006777648124329886
0.007042812366494649


#### MLPRegressor

In [43]:

parameters = {
    'solver': ['lbfgs'],  # Wrap the single value in a list
    'alpha': [0],
    'hidden_layer_sizes': [(100), (10, 10), (10, 20)],
    'activation': ['identity', 'logistic', 'relu'],
    'max_iter': [1000]
}

_model = MLPRegressor()
my_mlp = GridSearchCV(_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train, y_train)

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

In [44]:
my_mlp.best_estimator_

MLPRegressor(activation='logistic', alpha=0, hidden_layer_sizes=100,
             max_iter=1000, solver='lbfgs')

In [46]:
mlp_model = MLPRegressor(solver='lbfgs', alpha=0, hidden_layer_sizes=100, activation='logistic', max_iter=1000).fit(X_train, y_train)
print(mlp_model.score(X_train,y_train),mlp_model.score(X_test,y_test))

print(mean_squared_error(y_train,mlp_model.predict(X_train)))
print(mean_squared_error(y_test,mlp_model.predict(X_test)))

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.021573693714643905 0.019220101625834762
0.006911683807327194
0.007107598126909796


In [47]:
parameters = {
    'solver': ['sgd', 'adam'],
    'alpha': [0],
    'hidden_layer_sizes': [(100), (10, 20), (10, 10)],
    'activation': ['identity', 'logistic', 'relu'],
    'max_iter': [1000],
    'learning_rate_init': [0.01, 0.1],
    'batch_size': [20, 50]
}
_model=MLPRegressor()
my_mlp1=GridSearchCV(_model,param_grid=parameters, return_train_score=True, cv=5).fit(X_train,y_train)


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perce

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:339: RuntimeWarning: invalid value encountered in scalar multiply
  loss += (0.5 * self.alpha) * values / n_samples

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, wh

...

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:339: RuntimeWarning: invalid value encountered in scalar multiply
  loss += (0.5 * self.alpha) * values / n_samples
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y 

In [226]:
my_mlp1.best_estimator_

MLPRegressor(activation='logistic', alpha=0, batch_size=50,
             hidden_layer_sizes=(10, 10), learning_rate_init=0.1, max_iter=1000,
             solver='sgd')

In [ ]:
mlp1_model=MLPRegressor(solver='sgd',aplha=0,hidden_layer_sizes=(10,10),activation='logistic',max_iter=1000,learning_rate_init=0.1,batch_size=50).fit(X_train,y_train)
print(mlp1.score(X_train,y_train),mlp1.score(X_test,y_test))
print(mean_squared_error(y_train,mlp1_model.predict(X_train)))
print(mean_squared_error(y_test,mlp1_model.predict(X_test)))

## Problem 7

In [279]:
import pandas as pd, requests
headers = {'User-Agent': "ae161725@gmail.com"}

companyTickers = requests.get("https://www.sec.gov/files/company_tickers.json" , headers=headers   )


In [183]:
companyData = pd.DataFrame.from_dict(companyTickers.json(),  orient='index')
companyData['cik_str10'] = companyData['cik_str'].astype(str).str.zfill(10)

companyData

,cik_str,ticker,title,cik_str10
0,320193,AAPL,Apple Inc.,0000320193
1,789019,MSFT,MICROSOFT CORP,0000789019
2,1652044,GOOGL,Alphabet Inc.,0001652044
3,1018724,AMZN,AMAZON COM INC,0001018724
4,1045810,NVDA,NVIDIA CORP,0001045810
...,...,...,...,...
10691,19617,JPM-PM,JPMORGAN CHASE & CO,0000019617
10692,19617,AMJ,JPMORGAN CHASE & CO,0000019617
10693,1845123,IVCPU,Swiftmerge Acquisition Corp.,0001845123
10694,1845123,IVCPW,Swiftmerge Acquisition Corp.,0001845123


In [184]:
symbols=_df['symbol'].to_list()
len(set(symbols))
symbols=set(symbols)
symbols=list(symbols)
symbols.sort()

In [ ]:
import pandas as pd

# Initialize an empty DataFrame to hold the concatenated data
all_asset_data = pd.DataFrame()

for symb in symbols:
    _cik = companyData[companyData['ticker'] == symb]['cik_str10']
    
    # Check if CIK is available
    if not _cik.empty:
        cik = _cik.iloc[0]
        
        # Fetch company facts
        companyFacts = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json', headers=headers)
        
        # Check if the request is successful and has the expected structure
        if companyFacts.status_code == 200 and 'facts' in companyFacts.json().keys() and 'us-gaap' in companyFacts.json()['facts'].keys() and 'Assets' in companyFacts.json()['facts']['us-gaap'].keys():
            
            # Extract asset data and create DataFrame
            asset_data = companyFacts.json()['facts']['us-gaap']['Assets']['units']['USD']
            asset = pd.DataFrame.from_dict(asset_data)
            asset['symbol'] = symb
            
            # Concatenate directly to the existing DataFrame
            all_asset_data = pd.concat([all_asset_data, asset], ignore_index=True)
            
            print(f"Data processed successfully for symbol: {symb}")
        else:
            print(f"Failed to fetch data for symbol: {symb}")
    else:
        print(f"CIK not available for symbol: {symb}")

# Check if there is data
if not all_asset_data.empty:
    print("Data concatenated successfully.")
    # Further processing or analysis can be done with 'all_asset_data'
else:
    print("No data available.")

In [272]:
    print(all_asset_data)
    all_asset_data = all_asset_data[all_asset_data['form'].isin(['10-K', '10-Q'])]
    all_asset_data['filed1'] = pd.to_datetime(all_asset_data['filed'])
    all_asset_data['year'] = all_asset_data['filed1'].dt.year
    all_asset_data['month'] = all_asset_data['filed1'].dt.month
    all_asset_data['day'] = all_asset_data['filed1'].dt.day
    
    all_asset_data2 = all_asset_data.sort_values(by='filed1')
    all_asset_data2 = all_asset_data2.groupby(['symbol','year', 'month']).first().reset_index()
    
    features = ['year', 'month', 'val', 'symbol']
    all_asset_data3 = all_asset_data2[features].copy()
    all_asset_data3.rename(columns={'val': 'asset'}, inplace=True)
    

    print("Data processed successfully.")
    print(all_asset_data3)

Data processed successfully for symbol: A
Data processed successfully for symbol: AAL
Data processed successfully for symbol: AAPL
Data processed successfully for symbol: ABBV
Data processed successfully for symbol: ABNB
Data processed successfully for symbol: ABT
Data processed successfully for symbol: ACGL
Data processed successfully for symbol: ACN
Data processed successfully for symbol: ADBE
Data processed successfully for symbol: ADI
Data processed successfully for symbol: ADM
Data processed successfully for symbol: ADP
Data processed successfully for symbol: ADSK
Data processed successfully for symbol: AEE
Data processed successfully for symbol: AEP
Data processed successfully for symbol: AES
Data processed successfully for symbol: AFL
Data processed successfully for symbol: AIG
Data processed successfully for symbol: AIZ
Data processed successfully for symbol: AJG
Data processed successfully for symbol: AKAM
Data processed successfully for symbol: ALB
Data processed successfully

Data processed successfully for symbol: FLT
Data processed successfully for symbol: FMC
Data processed successfully for symbol: FOX
Data processed successfully for symbol: FOXA
Data processed successfully for symbol: FRT
Data processed successfully for symbol: FSLR
Data processed successfully for symbol: FTNT
Data processed successfully for symbol: FTV
Data processed successfully for symbol: GD
Data processed successfully for symbol: GE
Data processed successfully for symbol: GEN
Data processed successfully for symbol: GILD
Data processed successfully for symbol: GIS
Data processed successfully for symbol: GL
Data processed successfully for symbol: GLW
Data processed successfully for symbol: GM
Data processed successfully for symbol: GNRC
Data processed successfully for symbol: GOOG
Data processed successfully for symbol: GOOGL
Data processed successfully for symbol: GPC
Data processed successfully for symbol: GPN
Data processed successfully for symbol: GRMN
Data processed successfully

Data processed successfully for symbol: PODD
Data processed successfully for symbol: POOL
Data processed successfully for symbol: PPG
Data processed successfully for symbol: PPL
Data processed successfully for symbol: PRU
Data processed successfully for symbol: PSA
Data processed successfully for symbol: PSX
Data processed successfully for symbol: PTC
Data processed successfully for symbol: PWR
Data processed successfully for symbol: PXD
Data processed successfully for symbol: PYPL
Data processed successfully for symbol: QCOM
Data processed successfully for symbol: QRVO
Data processed successfully for symbol: RCL
Data processed successfully for symbol: REG
Data processed successfully for symbol: REGN
Data processed successfully for symbol: RF
Data processed successfully for symbol: RHI
Data processed successfully for symbol: RJF
Data processed successfully for symbol: RL
Data processed successfully for symbol: RMD
Data processed successfully for symbol: ROK
Data processed successfully 

In [250]:
_df

,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,prev_price,return,next_month_return,cum_ret6,cum_ret12
122,A,2010,1,2010-01-29,20.050072,18.079800,-0.005306,0.014345,-1.057651,-0.369864,20.040653,-0.097844,0.122369,0.207149,0.550332
123,A,2010,2,2010-02-26,22.503576,20.292213,0.006162,0.012060,0.795518,0.511000,18.079800,0.122369,0.093134,0.225078,1.268206
124,A,2010,3,2010-03-31,24.599428,22.182100,0.003922,0.009486,0.943496,0.413428,20.292213,0.093134,0.054376,0.235717,1.237475
125,A,2010,4,2010-04-30,25.937054,23.388283,0.002628,0.014793,0.484652,0.177654,22.182100,0.054376,-0.107557,0.465642,0.985761
126,A,2010,5,2010-05-28,23.147352,20.872721,-0.005176,0.032369,0.345697,-0.159907,23.388283,-0.107557,-0.121446,0.118949,0.775096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201349,ZTS,2022,8,2022-08-31,156.529999,154.786163,-0.006552,0.015242,0.131837,-0.429848,180.516296,-0.142536,-0.052642,-0.188785,-0.229898
201350,ZTS,2022,9,2022-09-30,148.289993,146.637955,-0.002436,0.016930,0.549872,-0.143875,154.786163,-0.052642,0.018952,-0.210869,-0.231280
201351,ZTS,2022,10,2022-10-31,150.779999,149.417023,0.001047,0.017897,-0.199821,0.058510,146.637955,0.018952,0.022284,-0.145965,-0.297473
201352,ZTS,2022,11,2022-11-30,154.139999,152.746674,0.001700,0.036468,-1.137458,0.046623,149.417023,0.022284,-0.049241,-0.094653,-0.300707


In [264]:
all_asset_data4 = all_asset_data3[(all_asset_data3['year'] >= 2010) & (all_asset_data3['year'] <= 2022)]
all_asset_data4.sort_values(by=['symbol', 'year', 'month'], inplace=True)

final_data = pd.merge(_df, all_asset_data4, how='left', on=['symbol', 'year', 'month'])

final_data1 = final_data[(final_data['year'] >= 2010) & (final_data['year'] <= 2022)]

# Group by 'symbol' and fill NaN values within each group using forward fill
final_data1 = final_data1.fillna(method='ffill').reset_index()

final_data1.drop(columns=['index'],inplace=True)
final_data1.head(50)


C:\Users\jampa\AppData\Local\Temp\ipykernel_14056\369482554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_asset_data4.sort_values(by=['symbol', 'year', 'month'], inplace=True)


,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,prev_price,return,next_month_return,cum_ret6,cum_ret12,asset
0,A,2010,1,2010-01-29,20.050072,18.079800,-0.005306,0.014345,-1.057651,-0.369864,20.040653,-0.097844,0.122369,0.207149,0.550332,NaN
1,A,2010,2,2010-02-26,22.503576,20.292213,0.006162,0.012060,0.795518,0.511000,18.079800,0.122369,0.093134,0.225078,1.268206,NaN
2,A,2010,3,2010-03-31,24.599428,22.182100,0.003922,0.009486,0.943496,0.413428,20.292213,0.093134,0.054376,0.235717,1.237475,7.574000e+09
3,A,2010,4,2010-04-30,25.937054,23.388283,0.002628,0.014793,0.484652,0.177654,22.182100,0.054376,-0.107557,0.465642,0.985761,7.574000e+09
4,A,2010,5,2010-05-28,23.147352,20.872721,-0.005176,0.032369,0.345697,-0.159907,23.388283,-0.107557,-0.121446,0.118949,0.775096,7.574000e+09
5,A,2010,6,2010-06-30,20.336195,18.337812,-0.005598,0.023612,-0.678421,-0.237074,20.872721,-0.121446,-0.017587,-0.084969,0.399803,7.612000e+09
6,A,2010,7,2010-07-30,19.978540,18.015299,-0.000649,0.020241,-0.095339,-0.032066,18.337812,-0.017587,-0.034372,-0.003568,0.202842,7.612000e+09
7,A,2010,8,2010-08-31,19.291845,17.396082,-0.001236,0.027381,1.112051,-0.045157,18.015299,-0.034372,0.237301,-0.142721,0.050233,7.612000e+09
8,A,2010,9,2010-09-30,23.869814,21.524183,0.010359,0.018911,0.418899,0.547784,17.396082,0.237301,0.042853,-0.029660,0.199065,7.612000e+09
9,A,2010,10,2010-10-29,24.892704,22.446566,0.002081,0.013007,-0.983643,0.160000,21.524183,0.042853,0.006322,-0.040264,0.406629,9.100000e+09


In [269]:
import pandas as pd

# Initialize an empty DataFrame to hold the concatenated data
all_liabilities_data = pd.DataFrame()

for symb in symbols:
    _cik = companyData[companyData['ticker'] == symb]['cik_str10']
    
    # Check if CIK is available
    if not _cik.empty:
        cik = _cik.iloc[0]
        
        # Fetch company facts
        companyFacts = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json', headers=headers)
        
        # Check if the request is successful and has the expected structure
        if companyFacts.status_code == 200 and 'facts' in companyFacts.json().keys() and 'us-gaap' in companyFacts.json()['facts'].keys() and 'Liabilities' in companyFacts.json()['facts']['us-gaap'].keys():
            
            # Extract liabilities data and create DataFrame
            liabilities_data = companyFacts.json()['facts']['us-gaap']['Liabilities']['units']['USD']
            liabilities = pd.DataFrame.from_dict(liabilities_data)
            liabilities['symbol'] = symb
            
            # Concatenate directly to the existing DataFrame
            all_liabilities_data = pd.concat([all_liabilities_data, liabilities], ignore_index=True)
            
            print(f"Data processed successfully for symbol: {symb}")
        else:
            print(f"Failed to fetch data for symbol: {symb}")
    else:
        print(f"CIK not available for symbol: {symb}")

# Check if there is data
if not all_liabilities_data.empty:
    print("Data concatenated successfully.")
    # Further processing or analysis can be done with 'all_liabilities_data'
else:
    print("No data available.")

Data processed successfully for symbol: A
Failed to fetch data for symbol: AAL
Data processed successfully for symbol: AAPL
Failed to fetch data for symbol: ABBV
Data processed successfully for symbol: ABNB
Failed to fetch data for symbol: ABT
Data processed successfully for symbol: ACGL
Failed to fetch data for symbol: ACN
Data processed successfully for symbol: ADBE
Failed to fetch data for symbol: ADI
Failed to fetch data for symbol: ADM
Data processed successfully for symbol: ADP
Failed to fetch data for symbol: ADSK
Failed to fetch data for symbol: AEE
Data processed successfully for symbol: AEP
Failed to fetch data for symbol: AES
Data processed successfully for symbol: AFL
Data processed successfully for symbol: AIG
Data processed successfully for symbol: AIZ
Data processed successfully for symbol: AJG
Data processed successfully for symbol: AKAM
Failed to fetch data for symbol: ALB
Data processed successfully for symbol: ALGN
Failed to fetch data for symbol: ALK
Data processed 

Data processed successfully for symbol: GEN
Failed to fetch data for symbol: GILD
Data processed successfully for symbol: GIS
Data processed successfully for symbol: GL
Data processed successfully for symbol: GLW
Data processed successfully for symbol: GM
Data processed successfully for symbol: GNRC
Data processed successfully for symbol: GOOG
Data processed successfully for symbol: GOOGL
Failed to fetch data for symbol: GPC
Data processed successfully for symbol: GPN
Failed to fetch data for symbol: GRMN
Data processed successfully for symbol: GS
Failed to fetch data for symbol: GWW
Data processed successfully for symbol: HAL
Data processed successfully for symbol: HAS
Data processed successfully for symbol: HBAN
Data processed successfully for symbol: HCA
Data processed successfully for symbol: HD
Data processed successfully for symbol: HES
Data processed successfully for symbol: HIG
Data processed successfully for symbol: HII
Data processed successfully for symbol: HLT
Data processe

Data processed successfully for symbol: RJF
Data processed successfully for symbol: RL
Data processed successfully for symbol: RMD
Failed to fetch data for symbol: ROK
Data processed successfully for symbol: ROL
Data processed successfully for symbol: ROP
Failed to fetch data for symbol: ROST
Failed to fetch data for symbol: RSG
Data processed successfully for symbol: RTX
Data processed successfully for symbol: RVTY
Failed to fetch data for symbol: SBAC
Data processed successfully for symbol: SBUX
Data processed successfully for symbol: SCHW
Failed to fetch data for symbol: SEDG
Data processed successfully for symbol: SEE
Failed to fetch data for symbol: SHW
Data processed successfully for symbol: SJM
Data processed successfully for symbol: SLB
Data processed successfully for symbol: SNA
Data processed successfully for symbol: SNPS
Data processed successfully for symbol: SO
Data processed successfully for symbol: SPG
Data processed successfully for symbol: SPGI
Failed to fetch data for

In [273]:
    print(all_liabilities_data)
    all_liabilities_data = all_liabilities_data[all_liabilities_data['form'].isin(['10-K', '10-Q'])]
    all_liabilities_data['filed1'] = pd.to_datetime(all_liabilities_data['filed'])
    all_liabilities_data['year'] = all_liabilities_data['filed1'].dt.year
    all_liabilities_data['month'] = all_liabilities_data['filed1'].dt.month
    all_liabilities_data['day'] = all_liabilities_data['filed1'].dt.day
    
    all_liabilities_data2 = all_liabilities_data.sort_values(by='filed1')
    all_liabilities_data2 = all_liabilities_data2.groupby(['symbol','year', 'month']).first().reset_index()
    
    features = ['year', 'month', 'val', 'symbol']
    all_liabilities_data3 = all_liabilities_data2[features].copy()
    all_liabilities_data3.rename(columns={'val': 'liabilities'}, inplace=True)
    

    print("Data processed successfully.")
    print(all_liabilities_data3)

              end          val                  accn      fy  fp    form  \
0      2008-10-31   4448000000  0001140361-09-022500  2009.0  Q3  10-Q/A   
1      2008-10-31   4448000000  0001047469-09-010861  2009.0  FY    10-K   
2      2008-10-31   4448000000  0001104659-10-047478  2009.0  FY     8-K   
3      2009-07-31   4084000000  0001140361-09-022500  2009.0  Q3  10-Q/A   
4      2009-10-31   5106000000  0001047469-09-010861  2009.0  FY    10-K   
...           ...          ...                   ...     ...  ..     ...   
35923  2022-12-31  10522000000  0001555280-23-000205  2023.0  Q2    10-Q   
35924  2022-12-31  10522000000  0001555280-23-000247  2023.0  Q3    10-Q   
35925  2023-03-31   9263000000  0001555280-23-000150  2023.0  Q1    10-Q   
35926  2023-06-30   9128000000  0001555280-23-000205  2023.0  Q2    10-Q   
35927  2023-09-30   9032000000  0001555280-23-000247  2023.0  Q3    10-Q   

            filed      frame symbol  
0      2009-10-05        NaN      A  
1      2009

C:\Users\jampa\AppData\Local\Temp\ipykernel_14056\3621000704.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_liabilities_data['filed1'] = pd.to_datetime(all_liabilities_data['filed'])
C:\Users\jampa\AppData\Local\Temp\ipykernel_14056\3621000704.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_liabilities_data['year'] = all_liabilities_data['filed1'].dt.year
C:\Users\jampa\AppData\Local\Temp\ipykernel_14056\3621000704.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [274]:
all_liabilities_data4=all_liabilities_data3[(all_liabilities_data3['year'] >= 2010) & (all_liabilities_data3['year'] <= 2022)]
all_liabilities_data4.sort_values(by=['symbol','year','month'])
final_data1 = pd.merge(final_data1, all_liabilities_data4, how='left', on=['symbol', 'year', 'month'])
final_data2 = final_data1[(final_data1['year'] >= 2010) & (final_data1['year'] <= 2022)]
final_data2 = final_data2.fillna(method='ffill').reset_index()

final_data2.drop(columns=['index'],inplace=True)
final_data2.head(50)


C:\Users\jampa\AppData\Local\Temp\ipykernel_14056\271665417.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_liabilities_data4=all_liabilities_data3[(final_data['year'] >= 2010) & (final_data['year'] <= 2022)]


,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,prev_price,return,next_month_return,cum_ret6,cum_ret12,asset,netIncome_x,netIncome_y,liabilities
0,A,2010,1,2010-01-29,20.050072,18.079800,-0.005306,0.014345,-1.057651,-0.369864,20.040653,-0.097844,0.122369,0.207149,0.550332,NaN,NaN,NaN,NaN
1,A,2010,2,2010-02-26,22.503576,20.292213,0.006162,0.012060,0.795518,0.511000,18.079800,0.122369,0.093134,0.225078,1.268206,NaN,NaN,NaN,NaN
2,A,2010,3,2010-03-31,24.599428,22.182100,0.003922,0.009486,0.943496,0.413428,20.292213,0.093134,0.054376,0.235717,1.237475,7.574000e+09,NaN,NaN,5.098000e+09
3,A,2010,4,2010-04-30,25.937054,23.388283,0.002628,0.014793,0.484652,0.177654,22.182100,0.054376,-0.107557,0.465642,0.985761,7.574000e+09,NaN,NaN,5.098000e+09
4,A,2010,5,2010-05-28,23.147352,20.872721,-0.005176,0.032369,0.345697,-0.159907,23.388283,-0.107557,-0.121446,0.118949,0.775096,7.574000e+09,NaN,NaN,5.098000e+09
5,A,2010,6,2010-06-30,20.336195,18.337812,-0.005598,0.023612,-0.678421,-0.237074,20.872721,-0.121446,-0.017587,-0.084969,0.399803,7.612000e+09,NaN,NaN,5.098000e+09
6,A,2010,7,2010-07-30,19.978540,18.015299,-0.000649,0.020241,-0.095339,-0.032066,18.337812,-0.017587,-0.034372,-0.003568,0.202842,7.612000e+09,NaN,NaN,5.098000e+09
7,A,2010,8,2010-08-31,19.291845,17.396082,-0.001236,0.027381,1.112051,-0.045157,18.015299,-0.034372,0.237301,-0.142721,0.050233,7.612000e+09,NaN,NaN,5.098000e+09
8,A,2010,9,2010-09-30,23.869814,21.524183,0.010359,0.018911,0.418899,0.547784,17.396082,0.237301,0.042853,-0.029660,0.199065,7.612000e+09,NaN,NaN,5.098000e+09
9,A,2010,10,2010-10-29,24.892704,22.446566,0.002081,0.013007,-0.983643,0.160000,21.524183,0.042853,0.006322,-0.040264,0.406629,9.100000e+09,NaN,NaN,5.098000e+09


In [275]:
import pandas as pd

# Initialize an empty DataFrame to hold the concatenated data
all_ac_data = pd.DataFrame()

for symb in symbols:
    _cik = companyData[companyData['ticker'] == symb]['cik_str10']
    
    # Check if CIK is available
    if not _cik.empty:
        cik = _cik.iloc[0]
        
        # Fetch company facts
        companyFacts = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json', headers=headers)
        
        # Check if the request is successful and has the expected structure
        if companyFacts.status_code == 200 and 'facts' in companyFacts.json().keys() and 'us-gaap' in companyFacts.json()['facts'].keys() and 'AssetsCurrent' in companyFacts.json()['facts']['us-gaap'].keys():
            
            # Extract ac data and create DataFrame
            ac_data = companyFacts.json()['facts']['us-gaap']['AssetsCurrent']['units']['USD']
            ac = pd.DataFrame.from_dict(ac_data)
            ac['symbol'] = symb
            
            # Concatenate directly to the existing DataFrame
            all_ac_data = pd.concat([all_ac_data, ac], ignore_index=True)
            
            print(f"Data processed successfully for symbol: {symb}")
        else:
            print(f"Failed to fetch data for symbol: {symb}")
    else:
        print(f"CIK not available for symbol: {symb}")

# Check if there is data
if not all_ac_data.empty:
    print("Data concatenated successfully.")
    # Further processing or analysis can be done with 'all_ac_data'
else:
    print("No data available.")

Data processed successfully for symbol: A
Data processed successfully for symbol: AAL
Data processed successfully for symbol: AAPL
Data processed successfully for symbol: ABBV
Data processed successfully for symbol: ABNB
Data processed successfully for symbol: ABT
Failed to fetch data for symbol: ACGL
Data processed successfully for symbol: ACN
Data processed successfully for symbol: ADBE
Data processed successfully for symbol: ADI
Data processed successfully for symbol: ADM
Data processed successfully for symbol: ADP
Data processed successfully for symbol: ADSK
Data processed successfully for symbol: AEE
Data processed successfully for symbol: AEP
Data processed successfully for symbol: AES
Failed to fetch data for symbol: AFL
Failed to fetch data for symbol: AIG
Failed to fetch data for symbol: AIZ
Data processed successfully for symbol: AJG
Data processed successfully for symbol: AKAM
Data processed successfully for symbol: ALB
Data processed successfully for symbol: ALGN
Data proce

Data processed successfully for symbol: FSLR
Data processed successfully for symbol: FTNT
Data processed successfully for symbol: FTV
Data processed successfully for symbol: GD
Data processed successfully for symbol: GE
Data processed successfully for symbol: GEN
Data processed successfully for symbol: GILD
Data processed successfully for symbol: GIS
Failed to fetch data for symbol: GL
Data processed successfully for symbol: GLW
Data processed successfully for symbol: GM
Data processed successfully for symbol: GNRC
Data processed successfully for symbol: GOOG
Data processed successfully for symbol: GOOGL
Data processed successfully for symbol: GPC
Data processed successfully for symbol: GPN
Data processed successfully for symbol: GRMN
Failed to fetch data for symbol: GS
Data processed successfully for symbol: GWW
Data processed successfully for symbol: HAL
Data processed successfully for symbol: HAS
Failed to fetch data for symbol: HBAN
Data processed successfully for symbol: HCA
Data 

Data processed successfully for symbol: PYPL
Data processed successfully for symbol: QCOM
Data processed successfully for symbol: QRVO
Data processed successfully for symbol: RCL
Failed to fetch data for symbol: REG
Data processed successfully for symbol: REGN
Failed to fetch data for symbol: RF
Data processed successfully for symbol: RHI
Failed to fetch data for symbol: RJF
Data processed successfully for symbol: RL
Data processed successfully for symbol: RMD
Data processed successfully for symbol: ROK
Data processed successfully for symbol: ROL
Data processed successfully for symbol: ROP
Data processed successfully for symbol: ROST
Data processed successfully for symbol: RSG
Data processed successfully for symbol: RTX
Data processed successfully for symbol: RVTY
Data processed successfully for symbol: SBAC
Data processed successfully for symbol: SBUX
Failed to fetch data for symbol: SCHW
Data processed successfully for symbol: SEDG
Data processed successfully for symbol: SEE
Data pro

In [276]:
    print(all_ac_data)
    all_ac_data = all_ac_data[all_ac_data['form'].isin(['10-K', '10-Q'])]
    all_ac_data['filed1'] = pd.to_datetime(all_ac_data['filed'])
    all_ac_data['year'] = all_ac_data['filed1'].dt.year
    all_ac_data['month'] = all_ac_data['filed1'].dt.month
    all_ac_data['day'] = all_ac_data['filed1'].dt.day
    
    all_ac_data2 = all_ac_data.sort_values(by='filed1')
    all_ac_data2 = all_ac_data2.groupby(['symbol','year', 'month']).first().reset_index()
    
    features = ['year', 'month', 'val', 'symbol']
    all_ac_data3 = all_ac_data2[features].copy()
    all_ac_data3.rename(columns={'val': 'currentAsset'}, inplace=True)
    

    print("Data processed successfully.")
    print(all_ac_data3)

              end         val                  accn      fy  fp    form  \
0      2008-10-31  3182000000  0001140361-09-022500  2009.0  Q3  10-Q/A   
1      2008-10-31  3182000000  0001047469-09-010861  2009.0  FY    10-K   
2      2008-10-31  3182000000  0001104659-10-047478  2009.0  FY     8-K   
3      2009-07-31  2897000000  0001140361-09-022500  2009.0  Q3  10-Q/A   
4      2009-10-31  3961000000  0001047469-09-010861  2009.0  FY    10-K   
...           ...         ...                   ...     ...  ..     ...   
45200  2022-12-31  7506000000  0001555280-23-000205  2023.0  Q2    10-Q   
45201  2022-12-31  7506000000  0001555280-23-000247  2023.0  Q3    10-Q   
45202  2023-03-31  6269000000  0001555280-23-000150  2023.0  Q1    10-Q   
45203  2023-06-30  6183000000  0001555280-23-000205  2023.0  Q2    10-Q   
45204  2023-09-30  6222000000  0001555280-23-000247  2023.0  Q3    10-Q   

            filed      frame symbol  
0      2009-10-05        NaN      A  
1      2009-12-21      

In [277]:
all_ac_data4=all_ac_data3[(all_ac_data3['year'] >= 2010) & (all_ac_data3['year'] <= 2022)]
all_ac_data4.sort_values(by=['symbol','year','month'])
final_data2 = pd.merge(final_data2, all_ac_data4, how='left', on=['symbol', 'year', 'month'])
final_data3 = final_data2[(final_data2['year'] >= 2010) & (final_data2['year'] <= 2022)]
final_data3 = final_data3.fillna(method='ffill').reset_index()

final_data3.drop(columns=['index'],inplace=True)
final_data3.head(50)


,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,prev_price,return,next_month_return,cum_ret6,cum_ret12,asset,netIncome_x,netIncome_y,liabilities,currentAsset
0,A,2010,1,2010-01-29,20.050072,18.079800,-0.005306,0.014345,-1.057651,-0.369864,20.040653,-0.097844,0.122369,0.207149,0.550332,NaN,NaN,NaN,NaN,NaN
1,A,2010,2,2010-02-26,22.503576,20.292213,0.006162,0.012060,0.795518,0.511000,18.079800,0.122369,0.093134,0.225078,1.268206,NaN,NaN,NaN,NaN,NaN
2,A,2010,3,2010-03-31,24.599428,22.182100,0.003922,0.009486,0.943496,0.413428,20.292213,0.093134,0.054376,0.235717,1.237475,7.574000e+09,NaN,NaN,5.098000e+09,3.961000e+09
3,A,2010,4,2010-04-30,25.937054,23.388283,0.002628,0.014793,0.484652,0.177654,22.182100,0.054376,-0.107557,0.465642,0.985761,7.574000e+09,NaN,NaN,5.098000e+09,3.961000e+09
4,A,2010,5,2010-05-28,23.147352,20.872721,-0.005176,0.032369,0.345697,-0.159907,23.388283,-0.107557,-0.121446,0.118949,0.775096,7.574000e+09,NaN,NaN,5.098000e+09,3.961000e+09
5,A,2010,6,2010-06-30,20.336195,18.337812,-0.005598,0.023612,-0.678421,-0.237074,20.872721,-0.121446,-0.017587,-0.084969,0.399803,7.612000e+09,NaN,NaN,5.098000e+09,3.961000e+09
6,A,2010,7,2010-07-30,19.978540,18.015299,-0.000649,0.020241,-0.095339,-0.032066,18.337812,-0.017587,-0.034372,-0.003568,0.202842,7.612000e+09,NaN,NaN,5.098000e+09,3.961000e+09
7,A,2010,8,2010-08-31,19.291845,17.396082,-0.001236,0.027381,1.112051,-0.045157,18.015299,-0.034372,0.237301,-0.142721,0.050233,7.612000e+09,NaN,NaN,5.098000e+09,3.961000e+09
8,A,2010,9,2010-09-30,23.869814,21.524183,0.010359,0.018911,0.418899,0.547784,17.396082,0.237301,0.042853,-0.029660,0.199065,7.612000e+09,NaN,NaN,5.098000e+09,3.961000e+09
9,A,2010,10,2010-10-29,24.892704,22.446566,0.002081,0.013007,-0.983643,0.160000,21.524183,0.042853,0.006322,-0.040264,0.406629,9.100000e+09,NaN,NaN,5.098000e+09,3.961000e+09


In [281]:
import pandas as pd

# Initialize an empty DataFrame to hold the concatenated data
all_lc_data = pd.DataFrame()

for symb in symbols:
    _cik = companyData[companyData['ticker'] == symb]['cik_str10']
    
    # Check if CIK is available
    if not _cik.empty:
        cik = _cik.iloc[0]
        
        # Fetch company facts
        companyFacts = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json', headers=headers)
        
        # Check if the request is successful and has the expected structure
        if companyFacts.status_code == 200 and 'facts' in companyFacts.json().keys() and 'us-gaap' in companyFacts.json()['facts'].keys() and 'LiabilitiesCurrent' in companyFacts.json()['facts']['us-gaap'].keys():
            
            # Extract lc data and create DataFrame
            lc_data = companyFacts.json()['facts']['us-gaap']['LiabilitiesCurrent']['units']['USD']
            lc = pd.DataFrame.from_dict(lc_data)
            lc['symbol'] = symb
            
            # Concatenate directly to the existing DataFrame
            all_lc_data = pd.concat([all_lc_data, lc], ignore_index=True)
            
            print(f"Data processed successfully for symbol: {symb}")
        else:
            print(f"Failed to fetch data for symbol: {symb}")
    else:
        print(f"CIK not available for symbol: {symb}")

# Check if there is data
if not all_lc_data.empty:
    print("Data concatenated successfully.")
    # Further processing or analysis can be done with 'all_lc_data'
else:
    print("No data available.")

Data processed successfully for symbol: A
Data processed successfully for symbol: AAL
Data processed successfully for symbol: AAPL
Data processed successfully for symbol: ABBV
Data processed successfully for symbol: ABNB
Data processed successfully for symbol: ABT
Failed to fetch data for symbol: ACGL
Data processed successfully for symbol: ACN
Data processed successfully for symbol: ADBE
Data processed successfully for symbol: ADI
Data processed successfully for symbol: ADM
Data processed successfully for symbol: ADP
Data processed successfully for symbol: ADSK
Data processed successfully for symbol: AEE
Data processed successfully for symbol: AEP
Data processed successfully for symbol: AES
Failed to fetch data for symbol: AFL
Failed to fetch data for symbol: AIG
Failed to fetch data for symbol: AIZ
Data processed successfully for symbol: AJG
Data processed successfully for symbol: AKAM
Data processed successfully for symbol: ALB
Data processed successfully for symbol: ALGN
Data proce

Data processed successfully for symbol: FSLR
Data processed successfully for symbol: FTNT
Data processed successfully for symbol: FTV
Data processed successfully for symbol: GD
Data processed successfully for symbol: GE
Data processed successfully for symbol: GEN
Data processed successfully for symbol: GILD
Data processed successfully for symbol: GIS
Failed to fetch data for symbol: GL
Data processed successfully for symbol: GLW
Data processed successfully for symbol: GM
Data processed successfully for symbol: GNRC
Data processed successfully for symbol: GOOG
Data processed successfully for symbol: GOOGL
Data processed successfully for symbol: GPC
Data processed successfully for symbol: GPN
Data processed successfully for symbol: GRMN
Failed to fetch data for symbol: GS
Data processed successfully for symbol: GWW
Data processed successfully for symbol: HAL
Data processed successfully for symbol: HAS
Failed to fetch data for symbol: HBAN
Data processed successfully for symbol: HCA
Data 

Data processed successfully for symbol: PYPL
Data processed successfully for symbol: QCOM
Data processed successfully for symbol: QRVO
Data processed successfully for symbol: RCL
Failed to fetch data for symbol: REG
Data processed successfully for symbol: REGN
Failed to fetch data for symbol: RF
Data processed successfully for symbol: RHI
Failed to fetch data for symbol: RJF
Data processed successfully for symbol: RL
Data processed successfully for symbol: RMD
Data processed successfully for symbol: ROK
Data processed successfully for symbol: ROL
Data processed successfully for symbol: ROP
Data processed successfully for symbol: ROST
Data processed successfully for symbol: RSG
Data processed successfully for symbol: RTX
Data processed successfully for symbol: RVTY
Data processed successfully for symbol: SBAC
Data processed successfully for symbol: SBUX
Failed to fetch data for symbol: SCHW
Data processed successfully for symbol: SEDG
Data processed successfully for symbol: SEE
Data pro

In [282]:
    print(all_lc_data)
    all_lc_data = all_lc_data[all_lc_data['form'].isin(['10-K', '10-Q'])]
    all_lc_data['filed1'] = pd.to_datetime(all_lc_data['filed'])
    all_lc_data['year'] = all_lc_data['filed1'].dt.year
    all_lc_data['month'] = all_lc_data['filed1'].dt.month
    all_lc_data['day'] = all_lc_data['filed1'].dt.day
    
    all_lc_data2 = all_lc_data.sort_values(by='filed1')
    all_lc_data2 = all_lc_data2.groupby(['symbol','year', 'month']).first().reset_index()
    
    features = ['year', 'month', 'val', 'symbol']
    all_lc_data3 = all_lc_data2[features].copy()
    all_lc_data3.rename(columns={'val': 'currentLiablities'}, inplace=True)
    

    print("Data processed successfully.")
    print(all_lc_data3)

              end         val                  accn      fy  fp    form  \
0      2008-10-31  1330000000  0001140361-09-022500  2009.0  Q3  10-Q/A   
1      2008-10-31  1330000000  0001047469-09-010861  2009.0  FY    10-K   
2      2008-10-31  1330000000  0001104659-10-047478  2009.0  FY     8-K   
3      2009-07-31   991000000  0001140361-09-022500  2009.0  Q3  10-Q/A   
4      2009-10-31  1123000000  0001047469-09-010861  2009.0  FY    10-K   
...           ...         ...                   ...     ...  ..     ...   
45143  2022-12-31  3167000000  0001555280-23-000205  2023.0  Q2    10-Q   
45144  2022-12-31  3167000000  0001555280-23-000247  2023.0  Q3    10-Q   
45145  2023-03-31  1915000000  0001555280-23-000150  2023.0  Q1    10-Q   
45146  2023-06-30  1769000000  0001555280-23-000205  2023.0  Q2    10-Q   
45147  2023-09-30  1608000000  0001555280-23-000247  2023.0  Q3    10-Q   

            filed      frame symbol  
0      2009-10-05        NaN      A  
1      2009-12-21      

In [283]:
all_lc_data4=all_lc_data3[(all_lc_data3['year'] >= 2010) & (all_lc_data3['year'] <= 2022)]
all_lc_data4.sort_values(by=['symbol','year','month'])
final_data3 = pd.merge(final_data3, all_lc_data4, how='left', on=['symbol', 'year', 'month'])
final_data4 = final_data3[(final_data3['year'] >= 2010) & (final_data3['year'] <= 2022)]
final_data4 = final_data4.fillna(method='ffill').reset_index()

final_data4.drop(columns=['index'],inplace=True)
final_data4.head(50)


,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,...,return,next_month_return,cum_ret6,cum_ret12,asset,netIncome_x,netIncome_y,liabilities,currentAsset,currentLiablities
0,A,2010,1,2010-01-29,20.050072,18.079800,-0.005306,0.014345,-1.057651,-0.369864,...,-0.097844,0.122369,0.207149,0.550332,NaN,NaN,NaN,NaN,NaN,NaN
1,A,2010,2,2010-02-26,22.503576,20.292213,0.006162,0.012060,0.795518,0.511000,...,0.122369,0.093134,0.225078,1.268206,NaN,NaN,NaN,NaN,NaN,NaN
2,A,2010,3,2010-03-31,24.599428,22.182100,0.003922,0.009486,0.943496,0.413428,...,0.093134,0.054376,0.235717,1.237475,7.574000e+09,NaN,NaN,5.098000e+09,3.961000e+09,2.549000e+09
3,A,2010,4,2010-04-30,25.937054,23.388283,0.002628,0.014793,0.484652,0.177654,...,0.054376,-0.107557,0.465642,0.985761,7.574000e+09,NaN,NaN,5.098000e+09,3.961000e+09,2.549000e+09
4,A,2010,5,2010-05-28,23.147352,20.872721,-0.005176,0.032369,0.345697,-0.159907,...,-0.107557,-0.121446,0.118949,0.775096,7.574000e+09,NaN,NaN,5.098000e+09,3.961000e+09,2.549000e+09
5,A,2010,6,2010-06-30,20.336195,18.337812,-0.005598,0.023612,-0.678421,-0.237074,...,-0.121446,-0.017587,-0.084969,0.399803,7.612000e+09,NaN,NaN,5.098000e+09,3.961000e+09,1.123000e+09
6,A,2010,7,2010-07-30,19.978540,18.015299,-0.000649,0.020241,-0.095339,-0.032066,...,-0.017587,-0.034372,-0.003568,0.202842,7.612000e+09,NaN,NaN,5.098000e+09,3.961000e+09,1.123000e+09
7,A,2010,8,2010-08-31,19.291845,17.396082,-0.001236,0.027381,1.112051,-0.045157,...,-0.034372,0.237301,-0.142721,0.050233,7.612000e+09,NaN,NaN,5.098000e+09,3.961000e+09,1.123000e+09
8,A,2010,9,2010-09-30,23.869814,21.524183,0.010359,0.018911,0.418899,0.547784,...,0.237301,0.042853,-0.029660,0.199065,7.612000e+09,NaN,NaN,5.098000e+09,3.961000e+09,1.123000e+09
9,A,2010,10,2010-10-29,24.892704,22.446566,0.002081,0.013007,-0.983643,0.160000,...,0.042853,0.006322,-0.040264,0.406629,9.100000e+09,NaN,NaN,5.098000e+09,3.961000e+09,1.123000e+09


In [284]:
final_data4.drop(columns=['netIncome_x','netIncome_y'],inplace=True)

In [285]:
final_data4

,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,prev_price,return,next_month_return,cum_ret6,cum_ret12,asset,liabilities,currentAsset,currentLiablities
0,A,2010,1,2010-01-29,20.050072,18.079800,-0.005306,0.014345,-1.057651,-0.369864,20.040653,-0.097844,0.122369,0.207149,0.550332,NaN,NaN,NaN,NaN
1,A,2010,2,2010-02-26,22.503576,20.292213,0.006162,0.012060,0.795518,0.511000,18.079800,0.122369,0.093134,0.225078,1.268206,NaN,NaN,NaN,NaN
2,A,2010,3,2010-03-31,24.599428,22.182100,0.003922,0.009486,0.943496,0.413428,20.292213,0.093134,0.054376,0.235717,1.237475,7.574000e+09,5.098000e+09,3.961000e+09,2.549000e+09
3,A,2010,4,2010-04-30,25.937054,23.388283,0.002628,0.014793,0.484652,0.177654,22.182100,0.054376,-0.107557,0.465642,0.985761,7.574000e+09,5.098000e+09,3.961000e+09,2.549000e+09
4,A,2010,5,2010-05-28,23.147352,20.872721,-0.005176,0.032369,0.345697,-0.159907,23.388283,-0.107557,-0.121446,0.118949,0.775096,7.574000e+09,5.098000e+09,3.961000e+09,2.549000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73641,ZTS,2022,8,2022-08-31,156.529999,154.786163,-0.006552,0.015242,0.131837,-0.429848,180.516296,-0.142536,-0.052642,-0.188785,-0.229898,1.390000e+10,9.190000e+09,6.930000e+09,3.051000e+09
73642,ZTS,2022,9,2022-09-30,148.289993,146.637955,-0.002436,0.016930,0.549872,-0.143875,154.786163,-0.052642,0.018952,-0.210869,-0.231280,1.390000e+10,9.190000e+09,6.930000e+09,3.051000e+09
73643,ZTS,2022,10,2022-10-31,150.779999,149.417023,0.001047,0.017897,-0.199821,0.058510,146.637955,0.018952,0.022284,-0.145965,-0.297473,1.390000e+10,9.190000e+09,6.930000e+09,3.051000e+09
73644,ZTS,2022,11,2022-11-30,154.139999,152.746674,0.001700,0.036468,-1.137458,0.046623,149.417023,0.022284,-0.049241,-0.094653,-0.300707,1.367400e+10,9.012000e+09,6.551000e+09,2.880000e+09


In [286]:
#final_data5['ROA']=final_data5['netIncome']/final_data5['asset']

In [293]:
final_data4['CR']=final_data4['currentAsset']/final_data4['currentLiablities']

In [294]:
final_data4['DR']=final_data4['liabilities']/final_data4['asset']

In [295]:
features=['Close','avrg_return_daily','std_return_daily','skew_return_daily','Sharpe','return','cum_ret6','cum_ret12','CR','DR']

In [296]:
X=final_data4[features]
y=final_data4['next_month_return']

In [297]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8,test_size=0.25)
_avrg=X_train.mean()
_std=X_train.std()
X_train=(X_train - _avrg)/ (_std)
X_test=(X_test - _avrg)/ (_std)


### Ridge

In [317]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Impute missing values in X_train
imputer = SimpleImputer(strategy='mean')  # You can choose other strategies as well
X_train_imputed = imputer.fit_transform(X_train)

# Define Ridge regression model
ridge_model = Ridge()

# Define the parameter grid for GridSearchCV
parameter = {'alpha': [a for a in range(1000, 200000, 100)]}

# Perform GridSearchCV with Ridge model
my_ridge = GridSearchCV(ridge_model, param_grid=parameter, return_train_score=True, cv=5).fit(X_train_imputed, y_train)


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: ['CR']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [318]:
my_ridge.best_estimator_

Ridge(alpha=1900)

In [325]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Impute missing values in X_train and X_test
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Replace infinite values with a large finite value
X_train_imputed = np.where(np.isfinite(X_train_imputed), X_train_imputed, 1e10)
X_test_imputed = np.where(np.isfinite(X_test_imputed), X_test_imputed, 1e10)

# Scale your features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Define Ridge regression model
ridge_model = Ridge(alpha=19)

# Fit the Ridge model
ridge_model.fit(X_train_scaled, y_train)

# Evaluate the model
print("Train R^2:", ridge_model.score(X_train_scaled, y_train))
print("Test R^2:", ridge_model.score(X_test_scaled, y_test))
print("Train MSE:", mean_squared_error(y_train, ridge_model.predict(X_train_scaled)))
print("Test MSE:", mean_squared_error(y_test, ridge_model.predict(X_test_scaled)))


Train R^2: 0.01617195743579647
Test R^2: 0.016398453176121097
Train MSE: 0.007058692555299072
Test MSE: 0.0067377570818543606


C:\Users\jampa\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: ['CR']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: ['CR']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


### Decision

In [331]:
X_train = np.where(np.isfinite(X_train), X_train, np.nan)
print(np.max(np.abs(X_train)))
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth': [3], 'min_samples_leaf': [6], 'min_samples_split': [20], 'max_leaf_nodes': [6]}
model = DecisionTreeRegressor()
my_dfr = GridSearchCV(model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train_scaled, y_train)
print("Best Parameters:", my_dfr.best_params_)
print("Best Score:", my_dfr.best_score_)


1.7976931348623157e+308


C:\Users\jampa\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1076: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count


Best Parameters: {'max_depth': 3, 'max_leaf_nodes': 6, 'min_samples_leaf': 6, 'min_samples_split': 20}
Best Score: 0.022652394119179763


In [336]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


C:\Users\jampa\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1076: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count


In [338]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth': [3],
    'min_samples_leaf': [6],
    'min_samples_split': [20],
    'max_leaf_nodes': [6]
}

_model = DecisionTreeRegressor()

# Try each parameter setting separately to identify the problematic one
for param_name, param_values in parameters.items():
    for param_value in param_values:
        param_set = {param_name: param_value}
        try:
            my_dfr = GridSearchCV(_model, param_grid=param_set, return_train_score=True, cv=5).fit(X_train, y_train)
            print(f"Parameter set {param_set} works without issues.")
        except Exception as e:
            print(f"Parameter set {param_set} raised an exception:")
            print(e)


Parameter set {'max_depth': 3} raised an exception:
Parameter grid for parameter 'max_depth' needs to be a list or a numpy array, but got 3 (of type int) instead. Single values need to be wrapped in a list with one element.
Parameter set {'min_samples_leaf': 6} raised an exception:
Parameter grid for parameter 'min_samples_leaf' needs to be a list or a numpy array, but got 6 (of type int) instead. Single values need to be wrapped in a list with one element.
Parameter set {'min_samples_split': 20} raised an exception:
Parameter grid for parameter 'min_samples_split' needs to be a list or a numpy array, but got 20 (of type int) instead. Single values need to be wrapped in a list with one element.
Parameter set {'max_leaf_nodes': 6} raised an exception:
Parameter grid for parameter 'max_leaf_nodes' needs to be a list or a numpy array, but got 6 (of type int) instead. Single values need to be wrapped in a list with one element.


In [339]:
my_dfr.best_estimator_

DecisionTreeRegressor(max_depth=3, max_leaf_nodes=6, min_samples_leaf=6,
                      min_samples_split=20)

In [342]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Check for NaN or Infinite Values in X_train
print("NaN values in X_train:", np.isnan(X_train).any())
print("Infinite values in X_train:", np.isinf(X_train).any())

# Handle Missing Values (if any)
# You can use a method like mean imputation
# X_train = X_train.fillna(X_train.mean())

# Scale or Normalize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Check Target Values (y_train)
print("NaN values in y_train:", np.isnan(y_train).any())
print("Infinite values in y_train:", np.isinf(y_train).any())

# Handle Outliers (if any)
# You can use winsorization or other outlier handling techniques

# Define the DecisionTreeRegressor parameters
parameters = {
    'max_depth': [3],
    'min_samples_leaf': [6],
    'min_samples_split': [20],
    'max_leaf_nodes': [6]
}

# Create the DecisionTreeRegressor model
_model = DecisionTreeRegressor()

# Use GridSearchCV for hyperparameter tuning
my_dfr = GridSearchCV(_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train_scaled, y_train)

# Evaluate the model on training and test sets
print("Training R-squared:", my_dfr.score(X_train_scaled, y_train))
print("Testing R-squared:", my_dfr.score(X_test, y_test))

# Print Mean Squared Error on training and test sets
print("Mean Squared Error on Training Set:", mean_squared_error(y_train, my_dfr.predict(X_train_scaled)))
print("Mean Squared Error on Test Set:", mean_squared_error(y_test, my_dfr.predict(X_test)))


NaN values in X_train: False
Infinite values in X_train: False
NaN values in y_train: False
Infinite values in y_train: False


C:\Users\jampa\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1076: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count


Training R-squared: 0.028577021524121782
Testing R-squared: -0.0002734376761412971
Mean Squared Error on Training Set: 0.006969689670912845
Mean Squared Error on Test Set: 0.006851961000118275


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


### random

In [347]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Check for NaN values in the original data
print("NaN values in X_train:", np.isnan(X_train).any())
print("NaN values in y_train:", np.isnan(y_train).any())

# Use SimpleImputer to fill NaN values with the mean of each column
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)

# Check for NaN or infinite values after imputation and scaling
print("NaN values in X_train_imputed:", np.isnan(X_train_imputed).any())
print("Infinite values in X_train_imputed:", np.isinf(X_train_imputed).any())
print("NaN values in X_train_scaled:", np.isnan(X_train_scaled).any())
print("Infinite values in X_train_scaled:", np.isinf(X_train_scaled).any())

# Define the parameter grid for RandomForestRegressor
parameters = {
    'n_estimators': [50, 60, 100],
    'max_depth': [3, 4, 6],
    'min_samples_leaf': [6, 20],
    'min_samples_split': [20, 50],
    'max_leaf_nodes': [6, 12]
}

# Create a RandomForestRegressor instance
rf_model = RandomForestRegressor()

# Use GridSearchCV for hyperparameter tuning
my_rfr = GridSearchCV(rf_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train_scaled, y_train)

# Evaluate the model on training and test sets
print("Training R-squared:", my_rfr.score(X_train_scaled, y_train))
print("Best parameters:", my_rfr.best_params_)


NaN values in X_train: False
NaN values in y_train: False
NaN values in X_train_imputed: False
Infinite values in X_train_imputed: False
NaN values in X_train_scaled: False
Infinite values in X_train_scaled: False


C:\Users\jampa\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: [8]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Training R-squared: 0.044416339458773546
Best parameters: {'max_depth': 6, 'max_leaf_nodes': 12, 'min_samples_leaf': 6, 'min_samples_split': 50, 'n_estimators': 50}


In [348]:
my_rfr.best_estimator_

RandomForestRegressor(max_depth=6, max_leaf_nodes=12, min_samples_leaf=6,
                      min_samples_split=50, n_estimators=50)

In [353]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Check for NaN values in the original data
print("NaN values in X_train:", np.isnan(X_train_finite).any())
print("NaN values in y_train:", np.isnan(y_train_finite).any())
print("NaN values in X_test:", np.isnan(X_test).any())  # Add this line

# Use SimpleImputer to fill NaN values with the mean of each column
imputer = SimpleImputer(strategy="mean")
X_train_imputed = imputer.fit_transform(X_train_finite)
X_test_imputed = imputer.transform(X_test)  # Use the same imputer for the test set

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Create and fit the RandomForestRegressor
rfr_model = RandomForestRegressor(n_estimators=5, max_depth=6, min_samples_leaf=6, min_samples_split=50, max_leaf_nodes=12)
rfr_model.fit(X_train_scaled, y_train_finite)

# Evaluate the model
print("Training R-squared:", rfr_model.score(X_train_scaled, y_train_finite))
print("Testing R-squared:", rfr_model.score(X_test_scaled, y_test))
print("Training MSE:", mean_squared_error(y_train_finite, rfr_model.predict(X_train_scaled)))
print("Testing MSE:", mean_squared_error(y_test, rfr_model.predict(X_test_scaled)))


NaN values in X_train: False
NaN values in y_train: False
NaN values in X_test: Close                False
avrg_return_daily    False
std_return_daily     False
skew_return_daily    False
Sharpe               False
return               False
cum_ret6             False
cum_ret12            False
CR                   False
DR                   False
dtype: bool


C:\Users\jampa\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: [8]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: [8]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Training R-squared: 0.042389442827793444
Testing R-squared: 0.0216392814548505
Training MSE: 0.006870589389960527
Testing MSE: 0.006701856947329547


### Neural

In [354]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

parameters = {
    'solver': ['lbfgs'],  # Wrap the single value in a list
    'alpha': [0],  # Wrap the single value in a list
    'hidden_layer_sizes': [(100), (10, 10), (10, 20)],
    'activation': ['identity', 'logistic', 'relu'],
    'max_iter': [1000]
}

_model = MLPRegressor()
my_mlp = GridSearchCV(_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train, y_train)


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1005: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\n

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1005: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1005: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anac

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1005: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (ma

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1005: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.p

In [355]:
my_mlp.best_estimator_

MLPRegressor(activation='logistic', alpha=0, hidden_layer_sizes=100,
             max_iter=1000, solver='lbfgs')

In [356]:
mlp_model = MLPRegressor(solver='lbfgs', alpha=0, hidden_layer_sizes=100, max_iter=1000).fit(X_train, y_train)
print(mlp_model.score(X_train, y_train), mlp_model.score(X_test, y_test))

print(mean_squared_error(y_train, mlp_model.predict(X_train)))
print(mean_squared_error(y_test, mlp_model.predict(X_test)))


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2


-inf -1452288139928737.8
inf
9948301455294.541


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1005: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:478: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\jampa\anacon

In [358]:
parameters = {
    'solver': ['sgd', 'adam'],
    'alpha': [0],
    'hidden_layer_sizes': [(100), (10, 20), (10, 10)],
    'activation': ['identity', 'logistic', 'relu'],
    'max_iter': [1000],
    'learning_rate_init': [0.01, 0.1],
    'batch_size': [20, 50]
}

_model = MLPRegressor()
my_mlp1 = GridSearchCV(_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train, y_train)


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:233: RuntimeWarning: invalid value encountered in multiply
  coef_grads[layer] += self.alpha * self.coefs_[layer]
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:339: RuntimeWarning: invalid value encountered in scalar multiply
  loss += (0.5 * self.alpha) * values / n_samples
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\ne

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:339: RuntimeWarning: invalid value encountered in scalar multiply
  loss += (0.5 * self.alpha) * values / n_samples
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarnin

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:233: RuntimeWarning: invalid value encountered in multiply
  coef_grads[layer] += self.alpha * self.coefs_[layer]
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning:

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  

...

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_stochastic_optimizers.py:285: RuntimeWarning: invalid value encountered in divide
  -self.learning_rate * m / (np.sqrt(v) + self.epsilon)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matm

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_stochastic_optimizers.py:285: RuntimeWarning: invalid value encountered in divide
  -self.learning_rate * m / (np.sqrt(v) + self.epsilon)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_stochastic_optimizers.py:276: RuntimeWarning: overflow encountered in square
  self.beta_2 * v + (1 - self.beta_2) * (grad**2)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_stochastic_optimizers.py:285: RuntimeWarning: invalid value encountered in divide
  -self.learning_rate * m / (np.sqrt(v) + self.epsilon)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_netw

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:233: RuntimeWarning: invalid value encountered in multiply
  coef_grads[layer] += self.alpha * self.coefs_[layer]
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterati

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.

In [359]:
my_mlp1.best_estimator_

MLPRegressor(activation='logistic', alpha=0, batch_size=50,
             hidden_layer_sizes=(10, 10), learning_rate_init=0.01,
             max_iter=1000)

In [363]:
mlp1_model=MLPRegressor(alpha=0,hidden_layer_sizes=(10,10),activation='logistic',max_iter=1000,learning_rate_init=0.01,batch_size=50).fit(X_train,y_train)
print(mlp1_model.score(X_train,y_train),mlp1_model.score(X_test,y_test))

print(mean_squared_error(y_train,mlp1_model.predict(X_train)))
print(mean_squared_error(y_test,mlp1_model.predict(X_test)))

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b


0.016760458553819912 0.0003230679190108221
0.007054470223467839
0.006847874884341943


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


In [364]:
cik=companyData.iloc[0,3]
type(cik)

str

In [366]:
companyFacts = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',headers=headers )

#review data
companyFacts.json().keys()

dict_keys(['cik', 'entityName', 'facts'])

In [367]:
companyFacts.json()['facts']['dei']['EntityPublicFloat']['description']

"The aggregate market value of the voting and non-voting common equity held by non-affiliates computed by reference to the price at which the common equity was last sold, or the average bid and asked price of such common equity, as of the last business day of the registrant's most recently completed second fiscal quarter."

In [368]:
companyFacts.json()['facts']['us-gaap'].keys()

dict_keys(['AccountsPayable', 'AccountsPayableCurrent', 'AccountsReceivableNetCurrent', 'AccruedIncomeTaxesCurrent', 'AccruedIncomeTaxesNoncurrent', 'AccruedLiabilities', 'AccruedLiabilitiesCurrent', 'AccruedMarketingCostsCurrent', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentsToAdditionalPaidInCapitalTaxEffectFromShareBasedCompensation', 'AdvertisingExpense', 'AllocatedShareBasedCompensationExpense', 'AllowanceForDoubtfulAccountsReceivableCurrent', 'AmortizationOfIntangibleAssets', 'AntidilutiveSecuritiesExcludedFromCompu

In [369]:
companyFacts.json()['facts']['us-gaap']['Assets'].keys()

dict_keys(['label', 'description', 'units'])

In [370]:
apple_asset=pd.DataFrame.from_dict(companyFacts.json()['facts']['us-gaap']['Assets']['units']['USD'])
_symb=companyData[ companyData['cik_str10']==cik ]['ticker']
symb=_symb[0]
apple_asset['symbol']=symb
apple_asset

,end,val,accn,fy,fp,form,filed,frame,symbol
0,2008-09-27,39572000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,NaN,AAPL
1,2008-09-27,39572000000,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN,AAPL
2,2008-09-27,36171000000,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,NaN,AAPL
3,2008-09-27,36171000000,0001193125-10-238044,2010,FY,10-K,2010-10-27,CY2008Q3I,AAPL
4,2009-06-27,48140000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I,AAPL
...,...,...,...,...,...,...,...,...,...
119,2022-09-24,352755000000,0000320193-23-000106,2023,FY,10-K,2023-11-03,CY2022Q3I,AAPL
120,2022-12-31,346747000000,0000320193-23-000006,2023,Q1,10-Q,2023-02-03,CY2022Q4I,AAPL
121,2023-04-01,332160000000,0000320193-23-000064,2023,Q2,10-Q,2023-05-05,CY2023Q1I,AAPL
122,2023-07-01,335038000000,0000320193-23-000077,2023,Q3,10-Q,2023-08-04,CY2023Q2I,AAPL


In [371]:
apple_liabilities=pd.DataFrame.from_dict(companyFacts.json()['facts']['us-gaap']['Liabilities']['units']['USD'])
apple_liabilities['symbol']=symb
apple_liabilities

,end,val,accn,fy,fp,form,filed,frame,symbol
0,2008-09-27,18542000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,NaN,AAPL
1,2008-09-27,18542000000,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN,AAPL
2,2008-09-27,13874000000,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,CY2008Q3I,AAPL
3,2009-06-27,22252000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I,AAPL
4,2009-09-26,26019000000,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN,AAPL
...,...,...,...,...,...,...,...,...,...
117,2022-09-24,302083000000,0000320193-23-000106,2023,FY,10-K,2023-11-03,CY2022Q3I,AAPL
118,2022-12-31,290020000000,0000320193-23-000006,2023,Q1,10-Q,2023-02-03,CY2022Q4I,AAPL
119,2023-04-01,270002000000,0000320193-23-000064,2023,Q2,10-Q,2023-05-05,CY2023Q1I,AAPL
120,2023-07-01,274764000000,0000320193-23-000077,2023,Q3,10-Q,2023-08-04,CY2023Q2I,AAPL


In [372]:
pd.DataFrame.from_dict(companyFacts.json()['facts']['us-gaap']['EarningsPerShareBasic']['units']['USD/shares'])

,start,end,val,accn,fy,fp,form,filed,frame
0,2006-10-01,2007-09-29,4.04,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN
1,2006-10-01,2007-09-29,4.04,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,CY2007
2,2007-09-30,2008-06-28,4.20,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,NaN
3,2008-03-30,2008-06-28,1.21,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2008Q2
4,2007-09-30,2008-09-27,5.48,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN
...,...,...,...,...,...,...,...,...,...
297,2022-09-25,2023-04-01,3.42,0000320193-23-000064,2023,Q2,10-Q,2023-05-05,NaN
298,2023-01-01,2023-04-01,1.53,0000320193-23-000064,2023,Q2,10-Q,2023-05-05,CY2023Q1
299,2022-09-25,2023-07-01,4.69,0000320193-23-000077,2023,Q3,10-Q,2023-08-04,NaN
300,2023-04-02,2023-07-01,1.27,0000320193-23-000077,2023,Q3,10-Q,2023-08-04,CY2023Q2


In [373]:
pd.DataFrame.from_dict(companyFacts.json()['facts']['us-gaap']['Revenues']['units']['USD'])

,start,end,val,accn,fy,fp,form,filed,frame
0,2015-09-27,2016-09-24,215639000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2016
1,2016-09-25,2016-12-31,78351000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2016Q4
2,2017-01-01,2017-04-01,52896000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q1
3,2017-04-02,2017-07-01,45408000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q2
4,2016-09-25,2017-09-30,229234000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017
5,2017-07-02,2017-09-30,52579000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q3
6,2017-10-01,2017-12-30,88293000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q4
7,2017-12-31,2018-03-31,61137000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018Q1
8,2018-04-01,2018-06-30,53265000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018Q2
9,2017-10-01,2018-09-29,265595000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018


In [374]:
pd.DataFrame.from_dict(companyFacts.json()['facts']['us-gaap']['AssetsCurrent']['units']['USD'])

,end,val,accn,fy,fp,form,filed,frame
0,2008-09-27,32311000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,NaN
1,2008-09-27,32311000000,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN
2,2008-09-27,30006000000,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,CY2008Q3I
3,2009-06-27,35170000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I
4,2009-09-26,36265000000,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN
...,...,...,...,...,...,...,...,...
117,2022-09-24,135405000000,0000320193-23-000106,2023,FY,10-K,2023-11-03,CY2022Q3I
118,2022-12-31,128777000000,0000320193-23-000006,2023,Q1,10-Q,2023-02-03,CY2022Q4I
119,2023-04-01,112913000000,0000320193-23-000064,2023,Q2,10-Q,2023-05-05,CY2023Q1I
120,2023-07-01,122659000000,0000320193-23-000077,2023,Q3,10-Q,2023-08-04,CY2023Q2I


In [375]:
pd.DataFrame.from_dict(companyFacts.json()['facts']['us-gaap']['LiabilitiesCurrent']['units']['USD'])

,end,val,accn,fy,fp,form,filed,frame
0,2008-09-27,14092000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,NaN
1,2008-09-27,14092000000,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN
2,2008-09-27,11361000000,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,CY2008Q3I
3,2009-06-27,16661000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I
4,2009-09-26,19282000000,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN
...,...,...,...,...,...,...,...,...
117,2022-09-24,153982000000,0000320193-23-000106,2023,FY,10-K,2023-11-03,CY2022Q3I
118,2022-12-31,137286000000,0000320193-23-000006,2023,Q1,10-Q,2023-02-03,CY2022Q4I
119,2023-04-01,120075000000,0000320193-23-000064,2023,Q2,10-Q,2023-05-05,CY2023Q1I
120,2023-07-01,124963000000,0000320193-23-000077,2023,Q3,10-Q,2023-08-04,CY2023Q2I


In [376]:
pd.DataFrame.from_dict(companyFacts.json()['facts']['us-gaap']['Revenues']['units']['USD'])

,start,end,val,accn,fy,fp,form,filed,frame
0,2015-09-27,2016-09-24,215639000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2016
1,2016-09-25,2016-12-31,78351000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2016Q4
2,2017-01-01,2017-04-01,52896000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q1
3,2017-04-02,2017-07-01,45408000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q2
4,2016-09-25,2017-09-30,229234000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017
5,2017-07-02,2017-09-30,52579000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q3
6,2017-10-01,2017-12-30,88293000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q4
7,2017-12-31,2018-03-31,61137000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018Q1
8,2018-04-01,2018-06-30,53265000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018Q2
9,2017-10-01,2018-09-29,265595000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018


In [377]:
companyFacts.json()['facts']['us-gaap']['SalesRevenueNet']['label']

'Revenue, Net (Deprecated 2018-01-31)'

In [378]:
pd.DataFrame.from_dict(companyFacts.json()['facts']['us-gaap']['SalesRevenueNet']['units']['USD'])

,start,end,val,accn,fy,fp,form,filed,frame
0,2006-10-01,2007-09-29,24578000000,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,CY2007
1,2006-10-01,2007-09-29,24006000000,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN
2,2007-09-30,2008-06-28,24584000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,NaN
3,2008-03-30,2008-06-28,7464000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2008Q2
4,2007-09-30,2008-09-27,37491000000,0001193125-10-238044,2010,FY,10-K,2010-10-27,CY2008
...,...,...,...,...,...,...,...,...,...
205,2017-10-01,2017-12-30,88293000000,0000320193-18-000007,2018,Q1,10-Q,2018-02-02,CY2017Q4
206,2017-10-01,2018-03-31,149430000000,0000320193-18-000070,2018,Q2,10-Q,2018-05-02,NaN
207,2017-12-31,2018-03-31,61137000000,0000320193-18-000070,2018,Q2,10-Q,2018-05-02,CY2018Q1
208,2017-10-01,2018-06-30,202695000000,0000320193-18-000100,2018,Q3,10-Q,2018-08-01,NaN


In [379]:
'NetIncomeLoss'

'NetIncomeLoss'

In [380]:
companyFacts.json()['facts']['us-gaap']['AssetsCurrent']['description']

'Sum of the carrying amounts as of the balance sheet date of all assets that are expected to be realized in cash, sold, or consumed within one year (or the normal operating cycle, if longer). Assets are probable future economic benefits obtained or controlled by an entity as a result of past transactions or events.'

In [381]:
companyFacts.json()['facts']['us-gaap']['EarningsPerShareBasic']['description']


'The amount of net income (loss) for the period per each share of common stock or unit outstanding during the reporting period.'

In [ ]:
# for share price we will take closing price

## Problem 8

In [382]:
_y=_df['next_month_return']
X_train,X_test,y_train,y_test=train_test_split(_df,_y,random_state=8,test_size=0.25)

In [383]:
_decile=X_train.groupby(['year','month'])['next_month_return'].describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
_decile.reset_index().drop(columns=['count','mean','std','min','max'],inplace=True)
_decile.reset_index()

,year,month,count,mean,std,min,10%,20%,30%,40%,50%,60%,70%,80%,90%,max
0,2010,1,311.0,0.043726,0.067680,-0.103129,-0.030129,-0.007188,0.009896,0.022731,0.035314,0.050722,0.068601,0.084570,0.120999,0.402289
1,2010,2,310.0,0.070544,0.070144,-0.223545,-0.000129,0.024305,0.038383,0.051873,0.064146,0.076538,0.091137,0.112272,0.146784,0.447819
2,2010,3,328.0,0.038165,0.084212,-0.190801,-0.053777,-0.014974,0.002544,0.016971,0.037680,0.050789,0.065693,0.086353,0.145953,0.530917
3,2010,4,338.0,-0.064902,0.062564,-0.270521,-0.133351,-0.109330,-0.092800,-0.077423,-0.065148,-0.054343,-0.043204,-0.027465,-0.002583,0.248939
4,2010,5,323.0,-0.053420,0.059950,-0.256732,-0.131072,-0.097591,-0.078647,-0.064000,-0.050336,-0.036117,-0.021721,-0.003402,0.013331,0.149206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2022,8,376.0,-0.093197,0.063382,-0.291853,-0.167320,-0.139237,-0.120786,-0.106662,-0.093080,-0.078505,-0.067533,-0.052507,-0.023162,0.366568
152,2022,9,367.0,0.095183,0.097766,-0.349332,-0.025305,0.019600,0.056639,0.075825,0.096055,0.111315,0.143338,0.167205,0.199393,0.486796
153,2022,10,368.0,0.075074,0.075227,-0.144326,-0.009603,0.019972,0.039638,0.057356,0.072254,0.088426,0.100750,0.123392,0.163858,0.406559
154,2022,11,373.0,-0.048980,0.057208,-0.367334,-0.118396,-0.085971,-0.067712,-0.056382,-0.047158,-0.034639,-0.018923,-0.004027,0.020512,0.076729


In [384]:
X_train=pd.merge(X_train,_decile, on=['year','month'],how='left')

In [385]:
def decile_rank(row):
    if row['next_month_return'] < row["10%"]:
        return 1
    elif row['next_month_return'] < row["20%"]:
        return 2
    elif row['next_month_return'] < row["30%"]:
        return 3
    elif row['next_month_return'] < row["40%"]:
        return 4
    elif row['next_month_return'] < row["50%"]:
        return 5
    elif row['next_month_return'] < row["60%"]:
        return 6
    elif row['next_month_return'] < row["70%"]:
        return 7
    elif row['next_month_return'] < row["80%"]:
        return 8
    elif row['next_month_return'] < row["90%"]:
        return 9
    elif row['next_month_return'] > row["90%"]:
        return 10
    else:
        return np.nan

In [386]:
X_train['decile_rank']=X_train.apply(decile_rank,axis=1)
X_train=X_train.dropna()

In [387]:
X_test=pd.merge(X_test,_decile,how='left',on=['year','month'])

In [388]:
X_test['decile_rank']=X_test.apply(decile_rank,axis=1)
X_test=X_test.dropna()

In [389]:
features=['Close','avrg_return_daily','std_return_daily','skew_return_daily','Sharpe','return','cum_ret6','cum_ret12']
y_train=X_train['decile_rank']
y_test=X_test['decile_rank']
X_train=X_train[features]
X_test=X_test[features]
y_train

0        9.0
1        4.0
2        4.0
3        9.0
4        8.0
        ... 
55229    6.0
55230    5.0
55231    1.0
55232    5.0
55233    3.0
Name: decile_rank, Length: 55217, dtype: float64

In [390]:
_avg = X_train.mean()
_std = X_train.std()
X_train = (X_train - _avg)/(_std)
X_test = (X_test- _avg)/(_std)


In [392]:
parameters = {
    'solver': ['lbfgs'],
    'alpha': [0],
    'hidden_layer_sizes': [(100), (10, 10), (10, 20)],
    'activation': ['identity', 'logistic', 'relu'],
    'max_iter': [1000]
}

_model = MLPRegressor()
my_mlp = GridSearchCV(_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train, y_train)


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITE

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITE

In [393]:
my_mlp.best_estimator_

MLPRegressor(activation='identity', alpha=0, hidden_layer_sizes=(10, 20),
             max_iter=1000, solver='lbfgs')

In [401]:
mlp_model=MLPRegressor(solver='lbfgs',alpha=0,hidden_layer_sizes=(10,20),activation='identity',max_iter=1000).fit(X_train,y_train)
print(mlp_model.score(X_train,y_train),mlp_model.score(X_test,y_test))

0.0009809063823009723 4.5056330231108e-05


In [402]:
print(mean_squared_error(y_train,mlp_model.predict(X_train)))
print(mean_squared_error(y_test,mlp_model.predict(X_test)))

8.270926892718107
8.255800659768823


In [403]:
parameters = {
    'solver': ['sgd', 'adam'],
    'alpha': [0],
    'hidden_layer_sizes': [(100), (10, 20), (10, 10)],
    'activation': ['identity', 'logistic', 'relu'],
    'max_iter': [1000],
    'learning_rate_init': [0.01, 0.1],
    'batch_size': [20, 50]
}

_model = MLPRegressor()
my_mlp1 = GridSearchCV(_model, param_grid=parameters, return_train_score=True, cv=5).fit(X_train, y_train)


C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:339: RuntimeWarning: invalid value encountered in scalar multiply
  loss += (0.5 * self.alpha) * values / n_samples
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in m

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: 

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:233: RuntimeWarning: invalid value encountere

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:339: RuntimeWarning: invalid value encountered in scalar multiply
  loss += (0.5 * self.alpha) * values / n_samples
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:175: RuntimeWarning: invalid value encountered in add
  activations[i + 1] += self.intercepts_[i]
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:233: RuntimeWarning: invalid value encountered in multiply
  coef

C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
40 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jampa\anaconda3\Lib\site-packages\sklearn\base.py", line 11

In [405]:
my_mlp1.best_estimator_

MLPRegressor(alpha=0, batch_size=50, hidden_layer_sizes=100,
             learning_rate_init=0.01, max_iter=1000, solver='sgd')

In [408]:
mlp1_model=MLPRegressor(solver='sgd',alpha=0,hidden_layer_sizes=100,max_iter=1000,learning_rate_init=0.01,batch_size=50).fit(X_train,y_train)
print(mlp1_model.score(X_train,y_train),mlp1_model.score(X_test,y_test))

0.0077242650822496595 0.0015158146622553659


In [409]:
print(mean_squared_error(y_train,mlp1_model.predict(X_train)))
print(mean_squared_error(y_test,mlp1_model.predict(X_test)))

8.215098303279762
8.243657825049366


In [410]:
import pandas as pd

# Initialize an empty DataFrame to hold the concatenated data
all_netIncome_data = pd.DataFrame()

for symb in symbols:
    _cik = companyData[companyData['ticker'] == symb]['cik_str10']
    
    # Check if CIK is available
    if not _cik.empty:
        cik = _cik.iloc[0]
        
        # Fetch company facts
        companyFacts = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json', headers=headers)
        
        # Check if the request is successful and has the expected structure
        if companyFacts.status_code == 200 and 'facts' in companyFacts.json().keys() and 'us-gaap' in companyFacts.json()['facts'].keys() and 'NetIncomeLoss' in companyFacts.json()['facts']['us-gaap'].keys():
            
            # Extract asset data and create DataFrame
            income_data = companyFacts.json()['facts']['us-gaap']['NetIncomeLoss']['units']['USD']
            income = pd.DataFrame.from_dict(income_data)
            income['symbol'] = symb
            
            # Concatenate directly to the existing DataFrame
            all_netIncome_data = pd.concat([all_netIncome_data, income], ignore_index=True)
            
            print(f"Data processed successfully for symbol: {symb}")
        else:
            print(f"Failed to fetch data for symbol: {symb}")
    else:
        print(f"CIK not available for symbol: {symb}")

# Check if there is data
if not all_netIncome_data.empty:
    print("Data concatenated successfully.")
    # Further processing or analysis can be done with 'all_asset_data'
else:
    print("No data available.")
#######################
print(all_netIncome_data)
all_netIncome_data = all_netIncome_data.loc[all_netIncome_data['form'].isin(['10-K', '10-Q'])]
all_netIncome_data['period'] = pd.to_datetime(all_netIncome_data['end']) - pd.to_datetime(all_netIncome_data['start'])
all_netIncome_data2 = all_netIncome_data[all_netIncome_data['period'] <= pd.Timedelta(days=92)]
all_netIncome_data2['filed'] = pd.to_datetime(all_netIncome_data2['filed'])
all_netIncome_data2['year'] = all_netIncome_data2['filed'].dt.year
all_netIncome_data2['month'] = all_netIncome_data2['filed'].dt.month
all_netIncome_data2['day']=all_netIncome_data2['filed'].dt.day
all_netIncome_data2 = all_netIncome_data2.groupby(['symbol','year', 'month']).first().reset_index()
features = ['year', 'month', 'val', 'symbol']
all_netIncome_data3 = all_netIncome_data2[features2].copy()
all_netIncome_data3.rename(columns={'val': 'netIncome'}, inplace=True)

print("Data processed successfully.")
print(all_netIncome_data3)
##########################
all_netIncome_data4 = all_netIncome_data3[(all_netIncome_data3['year'] >= 2010) & (all_netIncome_data3['year'] <= 2022)]

final_data1 = pd.merge(final_data1, all_netIncome_data4, how='left', on=['symbol', 'year', 'month'])
final_data2 = final_data1[(final_data1['year'] >= 2010) & (final_data1['year'] <= 2022)]

# Reset the index to avoid alignment issues
final_data2.reset_index(drop=True, inplace=True)

final_data2.ffill(inplace=True)
final_data2.head(50)
 

Data processed successfully for symbol: A
Data processed successfully for symbol: AAL
Data processed successfully for symbol: AAPL
Data processed successfully for symbol: ABBV
Data processed successfully for symbol: ABNB
Data processed successfully for symbol: ABT
Data processed successfully for symbol: ACGL
Data processed successfully for symbol: ACN
Data processed successfully for symbol: ADBE
Data processed successfully for symbol: ADI
Data processed successfully for symbol: ADM
Data processed successfully for symbol: ADP
Data processed successfully for symbol: ADSK
Data processed successfully for symbol: AEE
Data processed successfully for symbol: AEP
Data processed successfully for symbol: AES
Data processed successfully for symbol: AFL
Data processed successfully for symbol: AIG
Data processed successfully for symbol: AIZ
Data processed successfully for symbol: AJG
Data processed successfully for symbol: AKAM
Data processed successfully for symbol: ALB
Data processed successfully

Data processed successfully for symbol: FMC
Failed to fetch data for symbol: FOX
Failed to fetch data for symbol: FOXA
Data processed successfully for symbol: FRT
Data processed successfully for symbol: FSLR
Data processed successfully for symbol: FTNT
Failed to fetch data for symbol: FTV
Data processed successfully for symbol: GD
Data processed successfully for symbol: GE
Data processed successfully for symbol: GEN
Data processed successfully for symbol: GILD
Data processed successfully for symbol: GIS
Data processed successfully for symbol: GL
Data processed successfully for symbol: GLW
Data processed successfully for symbol: GM
Data processed successfully for symbol: GNRC
Data processed successfully for symbol: GOOG
Data processed successfully for symbol: GOOGL
Data processed successfully for symbol: GPC
Data processed successfully for symbol: GPN
Data processed successfully for symbol: GRMN
Data processed successfully for symbol: GS
Data processed successfully for symbol: GWW
Data 

Data processed successfully for symbol: PPG
Data processed successfully for symbol: PPL
Data processed successfully for symbol: PRU
Data processed successfully for symbol: PSA
Data processed successfully for symbol: PSX
Data processed successfully for symbol: PTC
Data processed successfully for symbol: PWR
Data processed successfully for symbol: PXD
Data processed successfully for symbol: PYPL
Data processed successfully for symbol: QCOM
Data processed successfully for symbol: QRVO
Data processed successfully for symbol: RCL
Data processed successfully for symbol: REG
Data processed successfully for symbol: REGN
Data processed successfully for symbol: RF
Data processed successfully for symbol: RHI
Data processed successfully for symbol: RJF
Data processed successfully for symbol: RL
Data processed successfully for symbol: RMD
Data processed successfully for symbol: ROK
Data processed successfully for symbol: ROL
Data processed successfully for symbol: ROP
Data processed successfully fo

C:\Users\jampa\AppData\Local\Temp\ipykernel_14056\3035898058.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_netIncome_data2['filed'] = pd.to_datetime(all_netIncome_data2['filed'])
C:\Users\jampa\AppData\Local\Temp\ipykernel_14056\3035898058.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_netIncome_data2['year'] = all_netIncome_data2['filed'].dt.year
C:\Users\jampa\AppData\Local\Temp\ipykernel_14056\3035898058.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

Data processed successfully.
       year  month  netIncome symbol
0      2017      3  121000000      A
1      2017      6   91000000      A
2      2017      9  124000000      A
3      2018      3  168000000      A
4      2018      5  164000000      A
...     ...    ...        ...    ...
20353  2022      8  512000000    ZTS
20354  2022     11  552000000    ZTS
20355  2023      5  595000000    ZTS
20356  2023      8  529000000    ZTS
20357  2023     11  529000000    ZTS

[20358 rows x 4 columns]


,symbol,year,month,Date,Close,adj_close,avrg_return_daily,std_return_daily,skew_return_daily,Sharpe,prev_price,return,next_month_return,cum_ret6,cum_ret12,asset,netIncome_x,netIncome_y,liabilities,netIncome
0,A,2010,1,2010-01-29,20.050072,18.079800,-0.005306,0.014345,-1.057651,-0.369864,20.040653,-0.097844,0.122369,0.207149,0.550332,NaN,NaN,NaN,NaN,NaN
1,A,2010,2,2010-02-26,22.503576,20.292213,0.006162,0.012060,0.795518,0.511000,18.079800,0.122369,0.093134,0.225078,1.268206,NaN,NaN,NaN,NaN,NaN
2,A,2010,3,2010-03-31,24.599428,22.182100,0.003922,0.009486,0.943496,0.413428,20.292213,0.093134,0.054376,0.235717,1.237475,7.574000e+09,NaN,NaN,5.098000e+09,NaN
3,A,2010,4,2010-04-30,25.937054,23.388283,0.002628,0.014793,0.484652,0.177654,22.182100,0.054376,-0.107557,0.465642,0.985761,7.574000e+09,NaN,NaN,5.098000e+09,NaN
4,A,2010,5,2010-05-28,23.147352,20.872721,-0.005176,0.032369,0.345697,-0.159907,23.388283,-0.107557,-0.121446,0.118949,0.775096,7.574000e+09,NaN,NaN,5.098000e+09,NaN
5,A,2010,6,2010-06-30,20.336195,18.337812,-0.005598,0.023612,-0.678421,-0.237074,20.872721,-0.121446,-0.017587,-0.084969,0.399803,7.612000e+09,NaN,NaN,5.098000e+09,NaN
6,A,2010,7,2010-07-30,19.978540,18.015299,-0.000649,0.020241,-0.095339,-0.032066,18.337812,-0.017587,-0.034372,-0.003568,0.202842,7.612000e+09,NaN,NaN,5.098000e+09,NaN
7,A,2010,8,2010-08-31,19.291845,17.396082,-0.001236,0.027381,1.112051,-0.045157,18.015299,-0.034372,0.237301,-0.142721,0.050233,7.612000e+09,NaN,NaN,5.098000e+09,NaN
8,A,2010,9,2010-09-30,23.869814,21.524183,0.010359,0.018911,0.418899,0.547784,17.396082,0.237301,0.042853,-0.029660,0.199065,7.612000e+09,NaN,NaN,5.098000e+09,NaN
9,A,2010,10,2010-10-29,24.892704,22.446566,0.002081,0.013007,-0.983643,0.160000,21.524183,0.042853,0.006322,-0.040264,0.406629,9.100000e+09,NaN,NaN,5.098000e+09,NaN
